In [2]:
import bz2
import json
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd

from six.moves import urllib
from time import time
from tqdm import tqdm


In [2]:
xmlfile='idwiki-20230720-pages-articles-multistream.xml'

In [3]:
ns = {'export-0.1': 'http://www.mediawiki.org/xml/export-0.10/'}
tags_to_skip = ["siteinfo"]


import xml.etree.ElementTree as ET

def parse_wiki_xml(filename):
    # tags_to_skip는 건너뛸 태그의 목록입니다. 예를 들어, "siteinfo" 태그 등을 건너뛰는 용도로 사용됩니다.
    skipping = ""  # 현재 건너뛰고 있는 태그를 저장하는 변수
    in_page = False  # 현재 'page' 태그 내부에 있는지 여부를 나타내는 변수

    # XML 파일을 파싱하면서 start와 end 이벤트를 반복적으로 처리합니다.
    for event, elem in ET.iterparse(filename, events=("start", "end",)):
        if event == "start":
            # tags_to_skip에 해당하는 태그가 있다면, 해당 태그를 건너뜁니다.
            for tag in tags_to_skip:
                if tag in elem.tag:
                    print("removing elem siteinfo")  # 현재 siteinfo 태그를 건너뛰고 있다는 메시지를 출력합니다.
                    skipping = tag
                    elem.clear()  # 건너뛰는 태그를 제거합니다.
                    break
            if in_page:
                continue
            if "page" in elem.tag:
                in_page = True  # 'page' 태그 내부에 들어왔으므로 in_page를 True로 설정합니다.
        elif event == "end":
            if skipping:
                # 현재 건너뛰고 있는 태그를 처리합니다.
                if skipping in elem.tag:
                    elem.clear()  # 건너뛰는 태그를 제거합니다.
                    skipping = ""  # skipping 변수를 초기화합니다.
            else:
                # 건너뛰는 태그가 아니라면 'page' 태그를 처리합니다.
                if "page" in elem.tag:
                    yield elem  # 'page' 태그를 반환합니다.
                    elem.clear()  # 'page' 태그를 처리했으므로 해당 태그를 제거합니다.
                    in_page = False  # 'page' 태그 내부를 빠져나왔으므로 in_page를 False로 설정합니다.
                    


pages = parse_wiki_xml(xmlfile)

In [4]:
def process_text(text):
    # Remove any text not normally visible
    text = re.sub(r"&amp;", "&", text)  # decode URL encoded chars
    text = re.sub(r"&nbsp;", " ", text) #html에서의 공백 제거
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<.*>", "", text)  # remove xml tags
    text = re.sub(r"<ref[^<]*</ref>", "", text)  # remove references <ref...> ... </ref>
    text = re.sub(r"<[^>]*>", "", text)  # remove xhtml tags
    text = re.sub(r"\[http:[^] ]*", "[", text)  # remove normal url, preserve visible text
    text = re.sub(r"\|thumb", "", text)  # remove images links, preserve caption
    text = re.sub(r"\|left", "", text)
    text = re.sub(r"\|right", "", text)
    text = re.sub(r"\|\d+px", "", text)
    text = re.sub(r"\[\[image:[^\[\]]*\|", "", text)
    text = re.sub(r"\[\[category:([^|\]]*)[^]]*\]\]", r"\1", text, flags=re.I)  # show categories without markup
    text = re.sub(r"\[\[[a-z\-]*:[^\]]*\]\]", "", text)  # remove links to other languages
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)  # remove wiki url, preserve visible text
    text = re.sub(r"{{[^}]*}}", "", text)  # remove {{icons}} and {tables}
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\[", "", text)  # remove [ and ]
    text = re.sub(r"\]", "", text)
    text = re.sub(r"\(", "", text)  # remove ( and )
    text = re.sub(r"\)", "", text)
    text = re.sub(r"&[^;]*;", " ", text)  # remove URL encoded chars
    text = re.sub(r"\"", "", text)  # remove ' and "
    text = re.sub(r"'", "", text)
    text = re.sub(r"_", "", text)  # remove _
    text = re.sub(r"\W+", " ", text)
    return text

In [31]:
import pandas as pd

df=pd.read_csv('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터\\rawdata\\20230810_Wiki_All_000003.csv')
df.head(15)

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
5,Analisis leksikal,2023-07-13T13:29:58Z,'''Analisis leksikal''' (bahasa Inggris: ''lex...
6,Bahasa Indonesia,2023-07-18T09:46:20Z,{{about|bahasa resmi sekaligus bahasa nasional...
7,Biologi,2023-06-29T09:58:39Z,{{multiple image|perrow=2|total_width = 250|im...
8,Bali,2023-07-16T13:30:00Z,{{Rujuk ke|Provinsi Bali|kegunaan lain}}{{dist...
9,Bahasa Bali,2023-07-19T04:53:09Z,{{Infobox Bahasa |name=Bali |nativename = {{na...


In [36]:
df['text'][6]

'{{about|bahasa resmi sekaligus bahasa nasional di Indonesia|ringkasan mengenai bahasa-bahasa yang ada di [[Indonesia]]|Bahasa di Indonesia}} {{Infobox Bahasa |name={{pagename}} |nativename=\'\'Bahasa Indonesia\'\' |states=[[Bahasa di Indonesia|Indonesia]] |region=[[Nusantara]] dan [[Orang Indonesia perantauan|wilayah diaspora Indonesia]] |speakers=[[Bahasa ibu|Penutur B1]]: {{sigfig|42,766|2}}&nbsp;juta (cacah jiwa 2010)<ref name="sensus2010"/> |rank=56 |familycolor=Austronesia |fam2=[[Rumpun bahasa Melayu-Polinesia|Melayu-Polinesia]] |fam3=[[Rumpun bahasa Melayu–Sumbawa|Melayu-Sumbawa]] |fam4=[[Bahasa Melayik|Melayik]] |fam5=[[Bahasa Melayu|Melayu]]<ref name="Abas 1987 230">{{cite journal |last=Abas |first=Husen |date=1987 |title=Indonesian as a Unifying Language of Wider Communication : A Historical and Sociolinguistic Perspective |url=https://openresearch-repository.anu.edu.au/handle/1885/145414 |url-status= |journal=Pacific Linguistics |volume=D-73 |issue=viii |pages=230 |doi=10.1

In [12]:
#20230811_text_정제
def process_test(text):
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text)
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text)
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'","", text)
    text = re.sub(r"\"", "", text)
    text = re.sub(r"\*", "", text)
    

    return text 


In [13]:
process_test(df['text'][])

'  |foto   |caption  Dari atas, kiri ke kanan Pantai Kuta Badung, Pura Besakih, Tanah Lot, GWK, Tirta Gangga, Monumen Bajra Sandhi, Tari Kecak di Uluwatu, Tari Tenun, dan Pantai Sanur.| |bendera  Flag of Bali.svg |lambang  Coat_of_arms_of_Bali.svg |peta  Bali in Indonesia (special marker).svg |koordinat  9º 0 - 7º 50 LS114º 0 - 116º 0 BT |motto  Bali dwipa jaya Jayalah Pulau Bali! |dasar hukum  UU No. 64 Tahun 1958 |hari jadi   |ibu kota  Kota Denpasar |kabupaten  8 |kota  1 |kecamatan  57 |kelurahan  80 |desa  636 |gubernurlink  Gubernur Bali |nama gubernur  I Wayan Koster |wakilgubernurlink  Wakil Gubernur Provinsi Bali |nama wakil gubernur  Tjokorda Oka Artha Ardana Sukawati |nama ketua DPRD  Nyoman Adi Wiryatama |nama sekretaris daerah  Dewa Made Indra |luas  5780,06 |ref luas   |penduduk  4304574 |tahun populasi  2022 |populasi ref   |kepadatan  747 |agama  Hindu 86,70% Islam 10,10% Kristen 2,50%— Protestan 1,67%—Katolik 0,83% Buddha 0,68% Konghucu 0,01% Agama asli Nusantara 0,01%

In [10]:
#20230811_반복문 사용
import re

def process_test(text):
    for _ in range(10):
        text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
       

    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*==", "", text)
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text)
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'","", text)
    text = re.sub(r"\*", "", text)


    return text

In [27]:
#20230814_text_정제
def process_test(text):
        
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    

    return text 


In [28]:
process_test(df['text']['6'])

KeyError: '6'

In [52]:
#20230816_text_정제
import re

def process_test(text):
        
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    #text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text) #wiki 점 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    #text = re.sub(r"\|.*? |\|\d+px", "", text) #remove | and image px
    text = re.sub(r'jmpl\|', '', text)
    text = re.sub(r'ka\|', '', text)
    text = re.sub(r'\d+px\|', '', text)
    text = re.sub(r'kiri\|', '', text)
    text = re.sub(r'thumb\|', '', text)
    text = re.sub(r'lurus\|', '', text)
    text = re.sub(r'left\|', '', text)
    text = re.sub(r'right\|', '', text)
    

    return text

In [14]:
process_test(df['text'][6])

'  |nativenameBahasa Indonesia |statesIndonesia |regionNusantara dan wilayah diaspora Indonesia |speakersPenutur B1:  juta (cacah jiwa 2010) |rank56 |familycolorAustronesia |fam2Melayu-Polinesia |fam3Melayu-Sumbawa |fam4Melayik |fam5Melayu |script Latin (Alfabet bahasa Indonesia)Braille bahasa Indonesia |nation ----  |minority (digunakan dalam misi menjaga perdamaian PBB) |agencyBadan Pengembangan dan Pembinaan Bahasa |mapIndonesian Language Map.svg |mapcaptionKeterangan: Wilayah Bahasa Indonesia dominan dipertuturkan dan sebagai bahasa resmi. Wilayah Bahasa Indonesia dituturkan oleh minoritas. |iso1id|iso2ind|iso3ind|signBahasa Isyarat Indonesia (BISINDO)|lingua31-MFA-ac|glottoindo1316|ancestorMelayik Purba|ancestor2Melayu Kuno|glottorefnameIndonesian|speakers2Penutur B2:  juta (cacah jiwa 2010) 300 juta (2022)|ancestor3Melayu Klasik (Riau-Lingga)|ancestor4Melayu Pramodern (Hindia Belanda atau Balai Pustaka)|standardsBahasa Indonesia Baku|dialectsBahasa Indonesia Gaul|pronunciation |n

In [76]:
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터'

In [3]:
import pandas as pd
df = pd.read_excel('../위키 데이터\\한국 관련 파일 파이프 기호 포함.xlsx')
df

,Unnamed: 0,title,date,text,cleansing_test,Sentence,Token,Sen_len,Sen_ID
0,465,Korea Utara,2023-02-10T17:18:33Z,{{Korea Utara infobox}} '''Republik Rakyat Dem...,Republik Rakyat Demokratik Korea atau Korea U...,Berkas:North Korea 1996 CIA map.jpg|Peta Korea...,"['Berkas', ':', 'North', 'Korea', '1996', 'CIA...",70,20230816_wikidata_Korea Utara-326
1,466,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...,(bahasa Korea Utara) (bahasa Korea Selatan) | ...,"['(', 'bahasa', 'Korea', 'Utara', ')', '(', 'b...",54,20230816_wikidata_Korea-1
2,467,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...,Kota terbesar | largest_settlement Seoul 서울 |...,"['Kota', 'terbesar', '|', 'largest_settlement'...",270,20230816_wikidata_Korea-2
3,499,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...,|left Gunung Baekdu Korea terletak di semenanj...,"['|left', 'Gunung', 'Baekdu', 'Korea', 'terlet...",13,20230816_wikidata_Korea-38
4,578,Hanja,2022-01-30T03:16:54Z,{{Infobox writing system |name=Hanja |type=[[L...,| hanja | rrHanja | mrHancha | yr Hanqca | ...,| hanja | rrHanja | mrHancha | yr Hanqca | nr...,"['|', 'hanja', '|', 'rrHanja', '|', 'mrHancha'...",71,20230816_wikidata_Hanja-1
...,...,...,...,...,...,...,...,...,...
1628,19606,Korea Terbuka 2023 (bulu tangkis),2023-07-19T11:42:27Z,{{Short description|Turnamen bulu tangkis di K...,|image |caption |dates 18–23 Juli |number...,Kodai Naraoka Chou Tien-chen Shi Yuqi Loh K...,"['Kodai', 'Naraoka', 'Chou', 'Tien-chen', 'Shi...",4502,20230816_wikidata_Korea Terbuka 2023 (bulu tan...
1629,19611,Kim Do-yeon (pemeran),2023-07-07T04:31:02Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Busan, Korea Selatan | other_...","| birth_place Busan, Korea Selatan | other_na...","['|', 'birth_place', 'Busan', ',', 'Korea', 'S...",53,20230816_wikidata_Kim Do-yeon (pemeran)-1
1630,19613,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,| genre | writer Jung Hyung-soo | director...,| genre | writer Jung Hyung-soo | director ...,"['|', 'genre', '|', 'writer', 'Jung', 'Hyung-s...",74,20230816_wikidata_Oasis (seri televisi Korea S...
1631,19616,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,| | yes | | | | | yes | | |headercolor...,| | yes | | | | | yes | | |headercolor ...,"['|', '|', 'yes', '|', '|', '|', '|', '|', 'ye...",344,20230816_wikidata_Templat:Infobox Korean-1


In [4]:
df[['Sentence']]

,Sentence
0,Berkas:North Korea 1996 CIA map.jpg|Peta Korea...
1,(bahasa Korea Utara) (bahasa Korea Selatan) | ...
2,Kota terbesar | largest_settlement Seoul 서울 |...
3,|left Gunung Baekdu Korea terletak di semenanj...
4,| hanja | rrHanja | mrHancha | yr Hanqca | nr...
...,...
1628,Kodai Naraoka Chou Tien-chen Shi Yuqi Loh K...
1629,"| birth_place Busan, Korea Selatan | other_na..."
1630,| genre | writer Jung Hyung-soo | director ...
1631,| | yes | | | | | yes | | |headercolor ...


In [55]:
#20230817_text_정제
#20230822_sentence_정제
#20230823_sentence_정제
import re

def process_test(test):
    test = re.sub(r"\|.*?   |\|\d+px","",test)
    test = re.sub(r"\|.*?  |\|\d+px","",test)
    test = re.sub(r"\|.*? |\|\d+px","",test)
    test = re.sub(r"\|.*?|\|\d+px","",test)
    test = re.sub(r"\d+px", "",test) #remove image px
    test = re.sub(r"[가-힣]+","",test) #remove 한글
    test = re.sub(r"[一-龥]+","",test) #remove 한자
    test = re.sub(r"\([^)]*\)","",test) #remove 소괄호, 소괄호 속 내용
    test = re.sub(r"http\S+", "", test) #remove http
    test = re.sub(r"\!","",test)
    test = re.sub(r'.*?\.jpg\s*', '', test) #jpg 앞쪽 문장 모두 제거
    test = re.sub(r'jpg', '', test)
    test = re.sub(r'Berkas:.*',"",test) #Berkas: 로 문장 시작 시 모두 제거
    test = re.sub(r'\s*section at the bottom of the .*$','',test)
    test = re.sub(r'-->',"",test)
    

    return test




In [32]:
#|birth 외에 것들 정제 시작
df['cleansing_sentence']= df['Sentence'].apply(process_test)
df

,Unnamed: 0,title,date,text,cleansing_test,Sentence,Token,Sen_len,Sen_ID,cleansing_sentence
0,465,Korea Utara,2023-02-10T17:18:33Z,{{Korea Utara infobox}} '''Republik Rakyat Dem...,Republik Rakyat Demokratik Korea atau Korea U...,Berkas:North Korea 1996 CIA map.jpg|Peta Korea...,"['Berkas', ':', 'North', 'Korea', '1996', 'CIA...",70,20230816_wikidata_Korea Utara-326,Korea Perang Korea Korea Selatan ...
1,466,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...,(bahasa Korea Utara) (bahasa Korea Selatan) | ...,"['(', 'bahasa', 'Korea', 'Utara', ')', '(', 'b...",54,20230816_wikidata_Korea-1,37 32 N 126 59 E largest_settlement_type
2,467,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...,Kota terbesar | largest_settlement Seoul 서울 |...,"['Kota', 'terbesar', '|', 'largest_settlement'...",270,20230816_wikidata_Korea-2,Kota terbesar Korea adalah wilayah semenanjun...
3,499,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...,|left Gunung Baekdu Korea terletak di semenanj...,"['|left', 'Gunung', 'Baekdu', 'Korea', 'terlet...",13,20230816_wikidata_Korea-38,Gunung Baekdu Korea terletak di semenanjung Ko...
4,578,Hanja,2022-01-30T03:16:54Z,{{Infobox writing system |name=Hanja |type=[[L...,| hanja | rrHanja | mrHancha | yr Hanqca | ...,| hanja | rrHanja | mrHancha | yr Hanqca | nr...,"['|', 'hanja', '|', 'rrHanja', '|', 'mrHancha'...",71,20230816_wikidata_Hanja-1,"Hanqca Hanca Hanja , atau Hanmun , yang kada..."
...,...,...,...,...,...,...,...,...,...,...
1628,19606,Korea Terbuka 2023 (bulu tangkis),2023-07-19T11:42:27Z,{{Short description|Turnamen bulu tangkis di K...,|image |caption |dates 18–23 Juli |number...,Kodai Naraoka Chou Tien-chen Shi Yuqi Loh K...,"['Kodai', 'Naraoka', 'Chou', 'Tien-chen', 'Shi...",4502,20230816_wikidata_Korea Terbuka 2023 (bulu tan...,Kodai Naraoka Chou Tien-chen Shi Yuqi Loh K...
1629,19611,Kim Do-yeon (pemeran),2023-07-07T04:31:02Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Busan, Korea Selatan | other_...","| birth_place Busan, Korea Selatan | other_na...","['|', 'birth_place', 'Busan', ',', 'Korea', 'S...",53,20230816_wikidata_Kim Do-yeon (pemeran)-1,"Busan, Korea Selatan Kim Do-yun Universitas Se..."
1630,19613,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,| genre | writer Jung Hyung-soo | director...,| genre | writer Jung Hyung-soo | director ...,"['|', 'genre', '|', 'writer', 'Jung', 'Hyung-s...",74,20230816_wikidata_Oasis (seri televisi Korea S...,Oasis adalah seri televisi Korea Selatan tah...
1631,19616,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,| | yes | | | | | yes | | |headercolor...,| | yes | | | | | yes | | |headercolor ...,"['|', '|', 'yes', '|', '|', '|', '|', '|', 'ye...",344,20230816_wikidata_Templat:Infobox Korean-1,


In [33]:
df.to_csv('20230823_파이프기호 정제 2차.csv', encoding = 'utf-8-sig', index = False)
#--20230823--

In [20]:
#|birth로 시작하는 문장 정제
df_filtered = df[df['Sentence'].str.startswith('| birth_place') | df['Sentence'].str.startswith('|birth_place')]
df_filtered.reset_index(drop = True, inplace=True)
df_filtered

,Unnamed: 0,title,date,text,cleansing_test,Sentence,Token,Sen_len,Sen_ID
0,1235,Han Myung-sook,2022-11-29T18:17:29Z,{{Infobox Officeholder |name = Han Myeong-sook...,"|birth_place Pyongyang, Korea Utara Han Myu...","|birth_place Pyongyang, Korea Utara Han Myun...","['|birth_place', 'Pyongyang', ',', 'Korea', 'U...",18,20230816_wikidata_Han Myung-sook-1
1,3101,Roh Tae-woo,2023-02-05T22:29:37Z,{{Infobox President |name = Roh Tae-woo |nativ...,"|birth_place Daegu, Provinsi Gyeongsang Utar...","|birth_place Daegu, Provinsi Gyeongsang Utara...","['|birth_place', 'Daegu', ',', 'Provinsi', 'Gy...",68,20230816_wikidata_Roh Tae-woo-1
2,3151,Choi Kyu-hah,2022-03-10T08:02:49Z,{{Infobox President |name = Choi Kyu-hah Gumay...,"|birth_place Wonju, Gangwon |death_date |d...","|birth_place Wonju, Gangwon |death_date |de...","['|birth_place', 'Wonju', ',', 'Gangwon', '|de...",37,20230816_wikidata_Choi Kyu-hah-1
3,3488,Gojong dari Korea Raya,2022-07-13T10:30:34Z,{{Infobox royalty |name =Gojong<br />光武帝<br />...,"|birth_place Unhyeongung, Hanseong, Dinasti J...","|birth_place Unhyeongung, Hanseong, Dinasti Jo...","['|birth_place', 'Unhyeongung', ',', 'Hanseong...",77,20230816_wikidata_Gojong dari Korea Raya-1
4,4472,Gong Yoo,2023-04-29T08:30:54Z,{{Infobox person |name = Gong Ji-cheol |image ...,"|birth_place Busan, Korea Selatan |occupatio...","|birth_place Busan, Korea Selatan |occupation...","['|birth_place', 'Busan', ',', 'Korea', 'Selat...",69,20230816_wikidata_Gong Yoo-1
...,...,...,...,...,...,...,...,...,...
628,19492,Sung Han-bin,2023-07-19T05:53:17Z,{{Infobox person | name = Sung Han-bin | image...,"| birth_place Chungcheongnam-do, Korea Selat...","| birth_place Chungcheongnam-do, Korea Selata...","['|', 'birth_place', 'Chungcheongnam-do', ',',...",32,20230816_wikidata_Sung Han-bin-1
629,19497,Park Gun-wook,2023-06-08T08:52:27Z,{{Infobox person | name = Park Gun-wook | imag...,"| birth_place Gyeonggi-do, Korea Selatan | d...","| birth_place Gyeonggi-do, Korea Selatan | de...","['|', 'birth_place', 'Gyeonggi-do', ',', 'Kore...",33,20230816_wikidata_Park Gun-wook-1
630,19575,Kim Sun-woong,2023-06-15T08:23:26Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Ansan, Korea Selatan | other_...","| birth_place Ansan, Korea Selatan | other_na...","['|', 'birth_place', 'Ansan', ',', 'Korea', 'S...",53,20230816_wikidata_Kim Sun-woong-1
631,19588,Kim Min-kyo,2023-06-20T05:04:56Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Seoul, Korea Selatan | medium...","| birth_place Seoul, Korea Selatan | medium ...","['|', 'birth_place', 'Seoul', ',', 'Korea', 'S...",69,20230816_wikidata_Kim Min-kyo-1


In [21]:
df_filtered['cleansing_sentence']= df_filtered['Sentence'].apply(process_test)
df_filtered


C:\Users\USER\AppData\Local\Temp\ipykernel_17848\4034796715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleansing_sentence']= df_filtered['Sentence'].apply(process_test)


,Unnamed: 0,title,date,text,cleansing_test,Sentence,Token,Sen_len,Sen_ID,cleansing_sentence
0,1235,Han Myung-sook,2022-11-29T18:17:29Z,{{Infobox Officeholder |name = Han Myeong-sook...,"|birth_place Pyongyang, Korea Utara Han Myu...","|birth_place Pyongyang, Korea Utara Han Myun...","['|birth_place', 'Pyongyang', ',', 'Korea', 'U...",18,20230816_wikidata_Han Myung-sook-1,"Pyongyang, Korea Utara Han Myung-sook adalah..."
1,3101,Roh Tae-woo,2023-02-05T22:29:37Z,{{Infobox President |name = Roh Tae-woo |nativ...,"|birth_place Daegu, Provinsi Gyeongsang Utar...","|birth_place Daegu, Provinsi Gyeongsang Utara...","['|birth_place', 'Daegu', ',', 'Provinsi', 'Gy...",68,20230816_wikidata_Roh Tae-woo-1,Roh Tae-woo adalah seorang mantan tentara Ang...
2,3151,Choi Kyu-hah,2022-03-10T08:02:49Z,{{Infobox President |name = Choi Kyu-hah Gumay...,"|birth_place Wonju, Gangwon |death_date |d...","|birth_place Wonju, Gangwon |death_date |de...","['|birth_place', 'Wonju', ',', 'Gangwon', '|de...",37,20230816_wikidata_Choi Kyu-hah-1,Choi Kyu-ha Gumayel adalah Presiden Korea Sel...
3,3488,Gojong dari Korea Raya,2022-07-13T10:30:34Z,{{Infobox royalty |name =Gojong<br />光武帝<br />...,"|birth_place Unhyeongung, Hanseong, Dinasti J...","|birth_place Unhyeongung, Hanseong, Dinasti Jo...","['|birth_place', 'Unhyeongung', ',', 'Hanseong...",77,20230816_wikidata_Gojong dari Korea Raya-1,Gojong atau Sang Kaisar Gwangmu adalah raja k...
4,4472,Gong Yoo,2023-04-29T08:30:54Z,{{Infobox person |name = Gong Ji-cheol |image ...,"|birth_place Busan, Korea Selatan |occupatio...","|birth_place Busan, Korea Selatan |occupation...","['|birth_place', 'Busan', ',', 'Korea', 'Selat...",69,20230816_wikidata_Gong Yoo-1,Gong Yoo yang lebih dikenal dengan nama pangg...
...,...,...,...,...,...,...,...,...,...,...
628,19492,Sung Han-bin,2023-07-19T05:53:17Z,{{Infobox person | name = Sung Han-bin | image...,"| birth_place Chungcheongnam-do, Korea Selat...","| birth_place Chungcheongnam-do, Korea Selata...","['|', 'birth_place', 'Chungcheongnam-do', ',',...",32,20230816_wikidata_Sung Han-bin-1,Sung Hanbin adalah seorang penyanyi Korea Sel...
629,19497,Park Gun-wook,2023-06-08T08:52:27Z,{{Infobox person | name = Park Gun-wook | imag...,"| birth_place Gyeonggi-do, Korea Selatan | d...","| birth_place Gyeonggi-do, Korea Selatan | de...","['|', 'birth_place', 'Gyeonggi-do', ',', 'Kore...",33,20230816_wikidata_Park Gun-wook-1,Korea Selatan Penyanyi Park Gun-wook adalah ...
630,19575,Kim Sun-woong,2023-06-15T08:23:26Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Ansan, Korea Selatan | other_...","| birth_place Ansan, Korea Selatan | other_na...","['|', 'birth_place', 'Ansan', ',', 'Korea', 'S...",53,20230816_wikidata_Kim Sun-woong-1,"Ansan, Korea Selatan Seon Ung, Kim Seon Ung Pe..."
631,19588,Kim Min-kyo,2023-06-20T05:04:56Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Seoul, Korea Selatan | medium...","| birth_place Seoul, Korea Selatan | medium ...","['|', 'birth_place', 'Seoul', ',', 'Korea', 'S...",69,20230816_wikidata_Kim Min-kyo-1,Kim Min-kyo adalah pemeran dan sutradara Kore...


In [10]:
df_filtered.loc[[632]]

,Unnamed: 0,title,date,text,cleansing_test,Sentence,Token,Sen_len,Sen_ID,cleansing_sentence
632,19611,Kim Do-yeon (pemeran),2023-07-07T04:31:02Z,{{family name hatnote|Kim||lang=Korean}} {{Inf...,"| birth_place Busan, Korea Selatan | other_...","| birth_place Busan, Korea Selatan | other_na...","['|', 'birth_place', 'Busan', ',', 'Korea', 'S...",53,20230816_wikidata_Kim Do-yeon (pemeran)-1,"Busan, Korea Selatan Kim Do-yun Universitas Se..."


In [176]:
df_filtered.to_csv('20230822_정규표현식을 통한 파이프라인 제거 시도 5차.csv', index =False)

In [189]:
df_filtered.to_csv('20230822_정규표현식을 통한 파이프라인 제거 시도 7차.csv', encoding ='utf-8-sig', index =False) #csv 파일 깨짐 방지 해결
#--20230822--

In [2]:
#--20230818--

import pandas as pd
df = pd.read_csv('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터\\20230816_not_filtered_wiki.csv')
df

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
771206,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
771207,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
771208,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
771209,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [5]:
df['cleansing_test']=df['text'].apply(process_test)

In [16]:
df

,title,date,text,cleansing_test
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...,Struktur heliks ganda DNA. Atom-atom pada stru...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...,| image Anwar Sadat cropped.jpg | caption A...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...,Azhar Mansor sedang mengemudikan kapal layar y...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...,Situs arkeologi di Australia Arkeologi adala...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...,Antropologi adalah ilmu tentang manusia. Ant...
...,...,...,...,...
771206,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...,pada | full_name Cesare Casadei | birth_dat...
771207,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...,"| birth_place Suresnes, Prancis | death_date..."
771208,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...,Srikandi Pemuda Pancasila adalah sebuah organi...
771209,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec...",


In [13]:
#title 안에 있는 제목의 순서 찾기
df_find = df[df['title'] == 'Alive (film 2015)'].index
df_find

Int64Index([450872], dtype='int64')

In [32]:
df_split = df.loc[400001:430001]
df_split.to_csv('20230816_not_filtered_wiki_6.5.csv', index = False)
#--20230818--

In [35]:
df[:1000]

,title,date,text,cleansing_test
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...,jmpl|ka|340px|Struktur heliks ganda DNA. Atom-...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...,| image Anwar Sadat cropped.jpg | caption A...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...,jmpl|Azhar Mansor sedang mengemudikan kapal la...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...,jmpl|Situs arkeologi di Australia Arkeologi ...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...,Antropologi adalah ilmu tentang manusia. Ant...
...,...,...,...,...
995,Layanan pesan singkat,2023-07-19T09:35:33Z,'''Layanan pesan singkat''' atau '''surat masa...,Layanan pesan singkat atau surat masa singkat ...
996,INRI,2023-07-01T07:34:51Z,[[Berkas:Inri.jpg|jmpl|ka|Fragmen lukisan Matt...,"jmpl|ka|Fragmen lukisan Matthias Grünewald, ~ ..."
997,Jumat Agung,2023-02-02T13:47:59Z,{{dablink|Artikel ini merupakan hari raya Kris...,Jumat Agung adalah Hari Jumat sebelum Minggu...
998,Kamis Putih,2023-06-17T16:42:05Z,[[Berkas:Semana Santa en Bilbao 1.JPG|ka|jmpl|...,ka|jmpl|270px|Arak-arakan Perjamuan Terakhir d...


In [34]:
cleansing_test_df = df[:1000]
cleansing_test_df.to_csv('20230816_wiki_sample_000001.csv')

In [5]:

def convert_to_df(pages):
    data = []
    t0 = time()
    for i, page in enumerate(pages):
        if i % 2000 == 1999:
            print("Read {}k articles. Elapsed time: {:.3f}s".format(int((i+1)/1000), time() - t0), end="\r")

        title = page.find('export-0.1:title', ns).text.lower()
        
        text = page.find('export-0.1:revision', ns).find('export-0.1:text', ns).text
        if not text:
            page.clear()
            continue

        text = process_text(text)

        words = text.split()
        
        total_words = len(words)
        total_long_words = len([w for w in words if len(w) > 3])
        if total_long_words < 15:
            page.clear()
            continue
        date = page.find('export-0.1:revision', ns).find('export-0.1:timestamp', ns).text
        text =  " ".join(words)
        data.append([title, date, text])

    # Membuat data frame dari list data
    columns = ['title', 'date', 'text']
    df = pd.DataFrame(data, columns=columns)
    return df


In [7]:
df_test=convert_to_df(pages)

removing elem siteinfo


In [9]:
df_test.to_csv('20230802_Wiki_All_000001.csv')

In [7]:
text_filename = os.path.splitext(xmlfile)[0]+".txt"
print("start writing wikipedia texts to {}".format(text_filename))
t0 = time()

with open(text_filename, "wb") as f:
    for i, text in enumerate(texts):
        f.write((json.dumps({"text": text}) + os.linesep).encode("utf-8"))
        if i % 1000 == 999:
            print("Done writing {}k pages. Elapsed time: {:.3f}s".format(int((i+1)/1000), time() - t0), end="\r")
print("Done writing wikipedia texts in {:.3f}s".format(time() - t0))

start writing wikipedia texts to idwiki-20230720-pages-articles-multistream.txt
removing elem siteinfo


list 와 title을 합친 새로운 dataframe 만들기

In [5]:

df_korea = pd.read_excel('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터\\정가휘Wiki_All_000001_list.xlsx')
df_korea

,Unnamed: 0,title,date,text
0,1402,korea selatan,2023-02-10T17:13:02Z,republik korea atau lebih dikenal sebagai kore...
1,1403,korea utara,2023-02-10T17:18:33Z,republik rakyat demokratik korea atau korea ut...
2,1479,korea,2023-06-11T06:06:24Z,imagemap locator map of korea svg officiallang...
3,2216,hanja,2022-01-30T03:16:54Z,hanja rr hanja mr hancha yr hanqca nr hanca ha...
4,2217,bahasa korea,2023-07-19T04:35:22Z,caption hanguk eo joseon eo korean ditulis han...
...,...,...,...,...
2126,680793,oasis (seri televisi korea selatan),2023-07-09T03:29:01Z,genre writer jung hyung soo director han hee s...
2127,681118,templat:infobox korean,2023-07-11T05:50:00Z,yes yes headercolor float collapse pic piccap ...
2128,681827,daftar katedral di korea utara,2023-07-18T11:17:41Z,jmpl ka katedral changchung di pyongyang korea...
2129,681873,banjir korea selatan 2023,2023-07-17T18:10:10Z,damages missing sembilan fatalities empat satu...


In [21]:
df_korea[['title']]

,title
0,korea selatan
1,korea utara
2,korea
3,hanja
4,bahasa korea
...,...
2126,oasis (seri televisi korea selatan)
2127,templat:infobox korean
2128,daftar katedral di korea utara
2129,banjir korea selatan 2023


In [13]:
df_korea.loc['title']

KeyError: 'title'

In [9]:
df_korea_list = df_korea['title'].tolist()
df_korea_list

['korea selatan',
 'korea utara',
 'korea',
 'hanja',
 'bahasa korea',
 'aksara han tradisional',
 'daftar tokoh korea utara',
 'kategori:korea',
 'daftar perguruan tinggi di korea selatan',
 'gwangju',
 'jeolla selatan',
 'templat:wilayah dan pembagian administratif korea selatan',
 'alih aksara korea yang disempurnakan',
 'tiga kerajaan korea',
 'daftar perusahaan di korea selatan',
 'perang korea',
 'kategori:perang korea',
 'templat:nama korea',
 'kim woo-choong',
 'chaebol',
 'busan',
 'tim nasional sepak bola korea selatan',
 'orang korea di tiongkok',
 'daftar presiden korea selatan',
 'semenanjung korea',
 'korean air',
 'daftar perdana menteri korea selatan',
 'han myung-sook',
 'bendera korea utara',
 'hari libur di korea selatan',
 'chuseok',
 'kalender korea',
 'bendera korea selatan',
 'pantai korea',
 'korean air lines penerbangan 007',
 'institut sains dan teknologi korea',
 'universitas korea',
 'krisis sandera korea selatan 2007',
 'hamster korea',
 'full house (seri t

In [11]:
df=pd.read_csv('rawdata\\20230810_Wiki_All_000003.csv')
df

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
773324,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
773325,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
773326,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
773327,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [12]:
df['title_lower'] = df['title'].str.lower()
df
#df_list = df_list.tolist()

,title,date,text,title_lower
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...,asam deoksiribonukleat
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...,anwar sadat
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...,azhar mansor
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...,arkeologi
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...,antropologi
...,...,...,...,...
773324,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...,cesare casadei
773325,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...,julie gayet
773326,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...,srikandi pemuda pancasila
773327,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec...",templat:kontak gerhana bulan total


In [13]:
df_filtered = df[df['title_lower'].isin(df_korea_list)]

In [14]:
df_filtered = df_filtered.drop(columns=['title_lower'])
df_filtered

,title,date,text
1411,Korea Selatan,2023-02-10T17:13:02Z,{{Korea Selatan infobox}} '''Republik Korea'''...
1412,Korea Utara,2023-02-10T17:18:33Z,{{Korea Utara infobox}} '''Republik Rakyat Dem...
1488,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...
2236,Hanja,2022-01-30T03:16:54Z,{{Infobox writing system |name=Hanja |type=[[L...
2237,Bahasa Korea,2023-07-19T04:35:22Z,{{Infobox Bahasa |name=Bahasa Korea |nativenam...
...,...,...,...
771827,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...
772193,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...
772961,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...
773011,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...


In [57]:
df_filtered.to_csv('20230816_filtered_wiki.csv', index = False)

In [15]:
#df_korea['title']을 제외한 나머지
df_not_filtered = df[~df['title_lower'].isin(df_korea_list)]
df_not_filtered = df_not_filtered.drop(columns=['title_lower'])
df_not_filtered

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
773324,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
773325,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
773326,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
773327,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [16]:
df_not_filtered.to_csv('20230816_not_filtered_wiki.csv', index = False)

In [17]:
df_not_filtered

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
773324,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
773325,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
773326,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
773327,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [91]:
df_filtered['cleansing_test'] =df_filtered['text'].apply(process_test) #함수를 적용할 때는 .apply()를 사용

In [92]:
df_filtered.reset_index(drop=True, inplace=True) #reset_index()는 순서 배열, inplace 적용 시 바로 결과 나옴

In [93]:
df_filtered

,title,date,text,cleansing_test
0,Korea Selatan,2023-02-10T17:13:02Z,{{Korea Selatan infobox}} '''Republik Korea'''...,Republik Korea () atau lebih dikenal sebagai ...
1,Korea Utara,2023-02-10T17:18:33Z,{{Korea Utara infobox}} '''Republik Rakyat Dem...,Republik Rakyat Demokratik Korea atau Korea U...
2,Korea,2023-06-11T06:06:24Z,{{dablink|Ini adalah artikel tentang Korea sec...,(bahasa Korea Utara) (bahasa Korea Selatan) ...
3,Hanja,2022-01-30T03:16:54Z,{{Infobox writing system |name=Hanja |type=[[L...,| hanja | rrHanja | mrHancha | yr Hanqca | ...
4,Bahasa Korea,2023-07-19T04:35:22Z,{{Infobox Bahasa |name=Bahasa Korea |nativenam...,"|caption Hanguk-eo, Joseon-eo (Korean) ditu..."
...,...,...,...,...
2113,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,| genre | writer Jung Hyung-soo | director...
2114,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,| | yes | | | | | yes | | |headercolor...
2115,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...,"Katedral Changchung di Pyongyang, Korea Utara...."
2116,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...,) | damages | missing 9 | fatalities 41 | ...


In [4]:
#--20230822--
#목표: 20230816_not_filtered파일을 기반으로, 저번에 추출했던 2000rows, 이번에 뽑을 title list(7000rows)를 20230816_not_filtered에 대조하여 새로운 wiki파일 만들기. 
#즉, title list가 20230816_not_filtered에 해당되는 것과 안되는 것을 filtered, not_filtered.csv로 분리하기

#os.listdir은 해당 경로 파일들을 list형태로 변경

# title list(7000rows) 작업 시작
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터\\20230816_not_filtered_wiki_정가휘'
dir = os.listdir(path)
dir

['20230816_not_filtered_wiki_10_정가휘.xlsx',
 '20230816_not_filtered_wiki_1_정가휘.xlsx',
 '20230816_not_filtered_wiki_2_정가휘.xlsx',
 '20230816_not_filtered_wiki_3_정가휘.xlsx',
 '20230816_not_filtered_wiki_5_정가휘.xlsx',
 '20230816_not_filtered_wiki_7_정가휘.xlsx',
 '20230816_not_filtered_wiki_8_정가휘.xlsx',
 '20230816_not_filtered_wiki_9_정가휘.xlsx']

In [21]:
# 모든 'title' 값을 저장할 빈 리스트 생성
titles_list = []

# 각 파일에 대해 반복하여 처리
for filename in dir:
    if filename.endswith('.xlsx'):  # 엑셀 파일인지 확인
        file_path = os.path.join(path, filename)
        
        # 엑셀 파일 읽기
        df = pd.read_excel(file_path)
        
        # 'title' 컬럼의 값들을 리스트에 추가
        titles_list.extend(df['title'].tolist())

# 최종적으로 모든 'title' 값을 담은 리스트 출력
print(titles_list)

['Seo Bok', 'Aespa (grup musik)', 'Take (album)', 'Yang Se-hyung', 'Antenna (label rekaman)', 'AOMG', 'Walpurgis Night', 'Drippin', 'Marina Kim', 'Park Ji-sun', 'Run On', 'Labyrinth (album mini)', "I Can't Stop Me", 'Hwang Ui-jo', 'Good Friends', 'Snowdrop (seri televisi)', 'Blackswan', 'Black Mamba (lagu)', 'The Man Standing Next', 'Kwak Do-won', 'Steel Rain 2: Summit', 'Fatal Promise', 'Ri Yong-ho', 'The Third Way of Love', 'Baldomero L처pez', 'Red Shoes and the Seven Dwarfs', 'Helen Kim', 'Kang Kyung-wha', 'Bloom*Iz', 'Heart*Iz', 'Get Revenge', 'Lee Uk-bae', 'Paviliun Unifikasi', 'Paviliun Phanmun', "Shinbi's House", 'Choe Son-hui', 'Ri Pyong-chol', 'Hangyeore', 'Live On', 'Hush (seri televisi)', 'Mr. Queen', 'Ko Yong-hui', 'Phoenix 2020', "Please Don't Date Him", 'Miracle in Cell No. 7 (film 2019)', 'Sixth Sense (seri televisi)', 'Run (seri televisi)', 'Romantic Call Centre', 'On & Off', 'Oh! My Part, You', "Don't Be the First One!", 'The Dreamer (seri televisi)', 'Friday Joy Packag

In [23]:
len(titles_list)

7137

In [43]:
df = pd.read_csv('C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터\\20230816_not_filtered_wiki.csv')
df

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
771206,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
771207,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
771208,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
771209,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [44]:
titles_series = pd.Series(titles_list)

In [45]:
df_filtered = df[df['title'].isin(titles_series)]
df_filtered

,title,date,text
401,Konfusianisme,2023-07-17T03:00:00Z,{{tak netral}} {{rapikan}} {{Infobox Chinese |...
2019,Asia,2023-07-19T12:11:07Z,{{kegunaanlain}} {{Infobox continent |title = ...
2158,Kanji,2022-12-21T05:17:27Z,{{disambiginfo}} {{Infobox Writing system |nam...
2228,Aksara Han,2022-03-04T08:32:23Z,{{Infobox writing system |name=Aksara Han <br>...
2301,Biksu,2023-05-29T06:39:19Z,{{more citations needed|date=Januari 2023}} [[...
...,...,...,...
709655,Juvenile Justice,2022-11-13T06:54:00Z,{{Infobox television | name = Juvenile Justice...
709711,Focus (album mini),2022-10-28T10:13:45Z,{{Yatim|Oktober 2022}} {{Infobox album | name ...
709981,BAP Pisco (AMP-156),2023-02-05T08:46:02Z,{{orphan|Oktober 2022}} [[Berkas:BAP Pisco (as...
709996,It's Beautiful Now,2023-06-11T02:45:23Z,{{Infobox television | name = It's Beautiful N...


In [46]:
df_filtered.reset_index(drop=True, inplace = True)
df_filtered

,title,date,text
0,Konfusianisme,2023-07-17T03:00:00Z,{{tak netral}} {{rapikan}} {{Infobox Chinese |...
1,Asia,2023-07-19T12:11:07Z,{{kegunaanlain}} {{Infobox continent |title = ...
2,Kanji,2022-12-21T05:17:27Z,{{disambiginfo}} {{Infobox Writing system |nam...
3,Aksara Han,2022-03-04T08:32:23Z,{{Infobox writing system |name=Aksara Han <br>...
4,Biksu,2023-05-29T06:39:19Z,{{more citations needed|date=Januari 2023}} [[...
...,...,...,...
7088,Juvenile Justice,2022-11-13T06:54:00Z,{{Infobox television | name = Juvenile Justice...
7089,Focus (album mini),2022-10-28T10:13:45Z,{{Yatim|Oktober 2022}} {{Infobox album | name ...
7090,BAP Pisco (AMP-156),2023-02-05T08:46:02Z,{{orphan|Oktober 2022}} [[Berkas:BAP Pisco (as...
7091,It's Beautiful Now,2023-06-11T02:45:23Z,{{Infobox television | name = It's Beautiful N...


In [48]:
df_filtered.to_csv('20230822_filtered_wiki.csv', index = False)

In [2]:
df1 = pd.read_csv('20230822_filtered_wiki.csv')
df2 = pd.read_csv('20230816_cleansingtest_filtered_wiki.csv')


combined_df = pd.concat([df1, df2], ignore_index=True)

# 결과 확인
combined_df

FileNotFoundError: [Errno 2] No such file or directory: '20230822_filtered_wiki.csv'

In [59]:
combined_df['cleansing_test'] = combined_df['text'].apply(process_test)

In [60]:
combined_df

,title,date,text,cleansing_test
0,Konfusianisme,2023-07-17T03:00:00Z,{{tak netral}} {{rapikan}} {{Infobox Chinese |...,| suz Zyú-ka | j Jyu4-gaa1 | y Yùh-gāa |...
1,Asia,2023-07-19T12:11:07Z,{{kegunaanlain}} {{Infobox continent |title = ...,|population 4.641.054.775 (1) |density 87 ...
2,Kanji,2022-12-21T05:17:27Z,{{disambiginfo}} {{Infobox Writing system |nam...,"Zhuyin, Hanzi sederhana, Chu Nom, Aksara Khi..."
3,Aksara Han,2022-03-04T08:32:23Z,{{Infobox writing system |name=Aksara Han <br>...,"200px |piccap ""Aksara Han"" dalam bentuk tradi..."
4,Biksu,2023-05-29T06:39:19Z,{{more citations needed|date=Januari 2023}} [[...,Para bhikkhu sedang berdoa di Candi Borobudur...
...,...,...,...,...
9206,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,| genre | writer Jung Hyung-soo | director...
9207,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,| | yes | | | | | yes | | |headercolor...
9208,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...,"Katedral Changchung di Pyongyang, Korea Utara...."
9209,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...,) | damages | missing 9 | fatalities 41 | ...


In [61]:
combined_df.to_csv('20230822_filtered_wiki_sum.csv', index = False)

In [73]:
df_not_filtered = df[~df['title'].isin(titles_series)]
df_not_filtered

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
771206,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
771207,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
771208,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
771209,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [74]:
df_not_filtered.to_csv('20230822_not_filtered_wiki.csv', index = False)
#--20230822--

In [1]:
#--20230831-- 원천데이터(기사) 합치기

import pandas as pd
import numpy as np
import os

In [40]:
os.getcwd()

'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\원천데이터\\20230820_원천데이터(기사)'

In [51]:
path = 'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\원천데이터\\20230820_원천데이터(기사)'
os.chdir(path)

In [60]:
os.getcwd()

'C:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\기사 데이터\\원시 데이터\\원천데이터\\20230820_원천데이터(기사)'

In [53]:
os.listdir()

['20230824_원천데이터(기사)_문장분리테스트_1.csv',
 '20230824_원천데이터(기사)_문장분리테스트_2.csv',
 '20230824_원천데이터(기사)_문장분리테스트_3.csv',
 '20230824_원천데이터(기사)_문장분리테스트_4.csv',
 '20230824_원천데이터(기사)_문장분리테스트_5.csv',
 '20230824_원천데이터(기사)_문장분리테스트_6.csv',
 '20230824_원천데이터(기사)_문장분리테스트_7.csv']

In [54]:
file_list = os.listdir()

In [55]:
from tqdm import tqdm
df_all = pd.DataFrame()

for file_name in tqdm(file_list):
    if file_name.endswith('.csv'):
        
        new_df = pd.read_csv(file_name)

    df_all =pd.concat([df_all, new_df], axis=0)

100%|██████████| 7/7 [00:09<00:00,  1.40s/it]


In [56]:
df_all

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230809_newsdata_Korea_000015,C:/Users/USER/Desktop/news data/20230809_data/...,Menyusul Pengakuan Yoo Se Yoon Mengemudi Dalam...,Newspaper,Korea,2013-05-29,2023-08-09,20230809_newsdata_Korea_000015_sen000001,27,Setelah berita tentang pengakuan Yoo Se Yoon y...,Setelah berita tentang pengakuan Yoo Se Yoon y...,Setelah berita tentang pengakuan Yoo Se Yoon y...,"['Setelah', 'berita', 'tentang', 'pengakuan', ..."
1,20230809_newsdata_Korea_000015,C:/Users/USER/Desktop/news data/20230809_data/...,Menyusul Pengakuan Yoo Se Yoon Mengemudi Dalam...,Newspaper,Korea,2013-05-29,2023-08-09,20230809_newsdata_Korea_000015_sen000002,24,Setelah berita tentang pengakuan Yoo Se Yoon y...,"MBC menyatakan, ''Radio Star' akan tayang hari...","MBC menyatakan , ' ' Radio Star ' akan tayang ...","['MBC', 'menyatakan', ',', ""'"", ""'"", 'Radio', ..."
2,20230809_newsdata_Korea_000015,C:/Users/USER/Desktop/news data/20230809_data/...,Menyusul Pengakuan Yoo Se Yoon Mengemudi Dalam...,Newspaper,Korea,2013-05-29,2023-08-09,20230809_newsdata_Korea_000015_sen000003,12,Setelah berita tentang pengakuan Yoo Se Yoon y...,"​​Juga, rekaman yang direncanakan untuk hari i...","​​Juga , rekaman yang direncanakan untuk hari ...","['\u200b\u200bJuga', ',', 'rekaman', 'yang', '..."
3,20230809_newsdata_Korea_000015,C:/Users/USER/Desktop/news data/20230809_data/...,Menyusul Pengakuan Yoo Se Yoon Mengemudi Dalam...,Newspaper,Korea,2013-05-29,2023-08-09,20230809_newsdata_Korea_000015_sen000004,26,Setelah berita tentang pengakuan Yoo Se Yoon y...,Yoo Se Yoon juga dijadwalkan untuk rekaman MBC...,Yoo Se Yoon juga dijadwalkan untuk rekaman MBC...,"['Yoo', 'Se', 'Yoon', 'juga', 'dijadwalkan', '..."
4,20230809_newsdata_Korea_000015,C:/Users/USER/Desktop/news data/20230809_data/...,Menyusul Pengakuan Yoo Se Yoon Mengemudi Dalam...,Newspaper,Korea,2013-05-29,2023-08-09,20230809_newsdata_Korea_000015_sen000005,13,Setelah berita tentang pengakuan Yoo Se Yoon y...,"Namun, tidak jelas apakah jadwalnya akan lebih...","Namun , tidak jelas apakah jadwalnya akan lebi...","['Namun', ',', 'tidak', 'jelas', 'apakah', 'ja..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46452,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000196,6,"Syarifah Fadiyah Alkaff, remaja yang masih ...",Di bawah pohon mangga?',Di bawah pohon mangga ? ',"['Di', 'bawah', 'pohon', 'mangga', '?', ""'""]"
46453,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000198,15,"Syarifah Fadiyah Alkaff, remaja yang masih ...","'Saya nggak mau jual, ini satu-satunya harta k...","' Saya nggak mau jual , ini satu-satunya harta...","[""'"", 'Saya', 'nggak', 'mau', 'jual', ',', 'in..."
46454,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000199,12,"Syarifah Fadiyah Alkaff, remaja yang masih ...","Nggak tahu nanti kalau saya sudah mati,' kata ...","Nggak tahu nanti kalau saya sudah mati , ' kat...","['Nggak', 'tahu', 'nanti', 'kalau', 'saya', 's..."
46455,20230808_newsdata_Korea_008901,C:/Users/USER/Desktop/news data/20230808_data_...,"Kala Rumah Rusak dan Sumur Tercemar, Siswi SMP...",Newspaper,Korea,2023-08-03,2023-08-08,20230808_newsdata_Korea_008901_sen000200,7,"Syarifah Fadiyah Alkaff, remaja yang masih ...","Hafsah, usianya kini 90 tahun.","Hafsah , usianya kini 90 tahun .","['Hafsah', ',', 'usianya', 'kini', '90', 'tahu..."


In [57]:
df_all['Word_Count'].sum()

7065522

In [58]:
df_all.reset_index(drop=True, inplace=True)
df_all.to_csv('20230831_newsdata_원천데이터.csv', index=False, encoding='utf-8-sig')
#--20230831--

In [24]:
#--20230914--
#20230822_not_filtered_wiki 파일에서 효진님이 보내주신 파일과 겹치는 한국 관련 위키 추출
import pandas as pd
import numpy as np


In [25]:
df = pd.read_excel('C:\\Users\\USER\\Desktop\\wiki 한국-인도네시아 관련 표제어230912.xlsx')
df

,title,date,text,cleansing_text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 969,Unnamed: 970,Unnamed: 971,Unnamed: 972,Unnamed: 973,Unnamed: 974,Unnamed: 975,Unnamed: 976,Unnamed: 977,Unnamed: 978
0,Ulsan,2022-12-19T11:08:02Z,{{Infobox settlement <!-- modelled after [[Seo...,| translit_lang1_type2 Hanja | translit_lang...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chuncheon,2022-12-31T16:09:24Z,{{Infobox settlement | name = Chuncheon | nati...,| settlement_type Kota | translit_lang1 Kor...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Suwon,2023-02-15T02:21:44Z,{{Infobox settlement | name = Suwon | native_n...,| translit_lang1_info3 Suwon-si | translit_l...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cheongju,2022-01-23T18:10:48Z,[[Berkas:Cheongjuflagpole1.jpg|jmpl|200px|Sudu...,Sudut kota Cheongju Cheongju merupakan nama ko...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jeonju,2022-12-17T22:52:04Z,[[Berkas:JeondongCC.jpg|jmpl|200px|Gereja Kato...,Gereja Katolik Jeondong Jeonju merupakan nama ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Kategori:Gojoseon,2017-11-26T22:03:23Z,{{Cat main}} {{Commonscat|Gojoseon}} [[Kategor...,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,Kategori:Balhae,2014-03-30T20:48:53Z,"Tokoh, acara, dan lain-lain yang terkait denga...","Tokoh, acara, dan lain-lain yang terkait denga...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,Kategori:Goryeo,2013-04-06T13:29:49Z,"Tokoh, acara, dan lain-lain yang terkait denga...","Tokoh, acara, dan lain-lain yang terkait denga...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,Kategori:Penguasa Joseon,2019-06-30T00:06:19Z,{{Commonscat|Rulers of the Joseon Dynasty}} [[...,ja:Category:?앶??썹럨 ko:遺꾨쪟:議곗꽑????nl:Categor...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df = df.to_csv('230914한국관련_위키데이터_박효진.csv', index = False, encoding = 'utf-8-sig')

In [61]:
df = pd.read_csv('20230914한국관련_위키데이터_박효진.csv')
df

,title,date,text,cleansing_text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 969,Unnamed: 970,Unnamed: 971,Unnamed: 972,Unnamed: 973,Unnamed: 974,Unnamed: 975,Unnamed: 976,Unnamed: 977,Unnamed: 978
0,Ulsan,2022-12-19T11:08:02Z,{{Infobox settlement <!-- modelled after [[Seo...,| translit_lang1_type2 Hanja | translit_lang...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Chuncheon,2022-12-31T16:09:24Z,{{Infobox settlement | name = Chuncheon | nati...,| settlement_type Kota | translit_lang1 Kor...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Suwon,2023-02-15T02:21:44Z,{{Infobox settlement | name = Suwon | native_n...,| translit_lang1_info3 Suwon-si | translit_l...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cheongju,2022-01-23T18:10:48Z,[[Berkas:Cheongjuflagpole1.jpg|jmpl|200px|Sudu...,Sudut kota Cheongju Cheongju merupakan nama ko...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jeonju,2022-12-17T22:52:04Z,[[Berkas:JeondongCC.jpg|jmpl|200px|Gereja Kato...,Gereja Katolik Jeondong Jeonju merupakan nama ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,Kategori:Gojoseon,2017-11-26T22:03:23Z,{{Cat main}} {{Commonscat|Gojoseon}} [[Kategor...,,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
425,Kategori:Balhae,2014-03-30T20:48:53Z,"Tokoh, acara, dan lain-lain yang terkait denga...","Tokoh, acara, dan lain-lain yang terkait denga...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,Kategori:Goryeo,2013-04-06T13:29:49Z,"Tokoh, acara, dan lain-lain yang terkait denga...","Tokoh, acara, dan lain-lain yang terkait denga...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
427,Kategori:Penguasa Joseon,2019-06-30T00:06:19Z,{{Commonscat|Rulers of the Joseon Dynasty}} [[...,ja:Category:?앶??썹럨 ko:遺꾨쪟:議곗꽑????nl:Categor...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df_filtered = pd.read_csv('20230822_not_filtered_wiki.csv')
df_filtered

,title,date,text
0,Asam deoksiribonukleat,2023-06-12T15:37:14Z,[[Berkas:DNA Structure+Key+Labelled.pn NoBB.pn...
1,Anwar Sadat,2022-05-05T11:26:32Z,{{Infobox officeholder | native_name = {{lang|...
2,Azhar Mansor,2022-11-27T20:56:40Z,[[Berkas:Azhar_Mansor_Jalur_Gemilang.jpg|jmpl|...
3,Arkeologi,2023-06-01T13:13:38Z,{{Terjemah|Inggris}}{{bedakan|Anarkisme|Arsite...
4,Antropologi,2023-07-09T03:24:39Z,{{antropologi}} {{Zoologi}} '''Antropologi''' ...
...,...,...,...
764113,Cesare Casadei,2023-07-20T07:58:29Z,{{Infobox football biography | name = Cesare C...
764114,Julie Gayet,2023-07-20T08:56:52Z,{{Infobox person | name = Julie Gayet | image ...
764115,Srikandi Pemuda Pancasila,2023-07-20T09:02:13Z,Srikandi Pemuda Pancasila adalah sebuah organi...
764116,Templat:Kontak gerhana bulan total,2023-07-20T09:06:26Z,"{| class=""toccolours"" |{{navbar|Total lunar ec..."


In [62]:
df_wiki_filtered = df_filtered[df_filtered['title'].isin(df['title'])]

In [63]:
df_wiki_filtered

,title,date,text
52623,Gyeongju,2022-12-14T02:18:45Z,{{Infobox settlement | name = Gyeongju | nativ...
58583,Perjanjian Shimonoseki,2021-06-14T05:22:06Z,[[Berkas:Treaty_of_Shimonoseiki.jpg|jmpl|Hotel...
64187,Kim Simin,2022-12-01T04:19:26Z,{{Orphan|date=Desember 2022}} [[Berkas:Kim Si-...
65239,Perang Tiongkok-Jepang Pertama,2023-04-07T16:13:35Z,{{Infobox Military Conflict |conflict=Perang T...
149986,Buyeo Pung,2021-09-08T07:38:17Z,"'''Buyeo Pung''', dikenal sebagai {{nihongo|''..."
150109,Seong dari Balhae,2019-06-05T09:32:34Z,"'''Seong dari Balhae''', kadang-kadang disebut..."
150781,Siklus Wiracarita,2022-12-26T12:17:45Z,'''Siklus Wiracarita''' ({{lang-el|Ἐπικὸς Κύκλ...
156090,Yi Gu,2023-01-07T06:31:31Z,{{Infobox royalty |name = Yi Gu |native name =...
156259,Empat Komando Han,2022-07-12T07:48:42Z,"'''Empat Jun di Dinasti Han''' (漢四郡, 한사군) terd..."
156582,Kebijakan Sinar Matahari,2022-08-08T00:41:47Z,{{nofootnote}} {{Infobox Korean name |hangul=햇...


In [64]:
#정규표현식을 'text'에 적용한 'cleansing_test' column 생성
df_wiki_filtered['cleansing_test'] = df_wiki_filtered['text'].apply(process_test)

C:\Users\USER\AppData\Local\Temp\ipykernel_15164\2091363133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wiki_filtered['cleansing_test'] = df_wiki_filtered['text'].apply(process_test)


In [65]:
df_wiki_filtered

,title,date,text,cleansing_test
52623,Gyeongju,2022-12-14T02:18:45Z,{{Infobox settlement | name = Gyeongju | nativ...,large seven-story hospital complex on a slope ...
58583,Perjanjian Shimonoseki,2021-06-14T05:22:06Z,[[Berkas:Treaty_of_Shimonoseiki.jpg|jmpl|Hotel...,Shunpanrō tempat penandatanganan perjanjian]] ...
64187,Kim Simin,2022-12-01T04:19:26Z,{{Orphan|date=Desember 2022}} [[Berkas:Kim Si-...,Simin]] '''Kim Simin''' adalah seorang [[jend...
65239,Perang Tiongkok-Jepang Pertama,2023-04-07T16:13:35Z,{{Infobox Military Conflict |conflict=Perang T...,"Tiongkok-Jepang Pertama, peperangan besar dan ..."
149986,Buyeo Pung,2021-09-08T07:38:17Z,"'''Buyeo Pung''', dikenal sebagai {{nihongo|''...","'''Buyeo Pung''', dikenal sebagai {{nihongoHōs..."
150109,Seong dari Balhae,2019-06-05T09:32:34Z,"'''Seong dari Balhae''', kadang-kadang disebut...","'''Seong dari Balhae''', kadang-kadang disebut..."
150781,Siklus Wiracarita,2022-12-26T12:17:45Z,'''Siklus Wiracarita''' ({{lang-el|Ἐπικὸς Κύκλ...,'''Siklus Wiracarita''' adalah sebutan untuk ...
156090,Yi Gu,2023-01-07T06:31:31Z,{{Infobox royalty |name = Yi Gu |native name =...,= Gu semasa kecil = Kaisar Korea = Ahli waris ...
156259,Empat Komando Han,2022-07-12T07:48:42Z,"'''Empat Jun di Dinasti Han''' (漢四郡, 한사군) terd...",'''Empat Jun di Dinasti Han''' terdiri dari [...
156582,Kebijakan Sinar Matahari,2022-08-08T00:41:47Z,{{nofootnote}} {{Infobox Korean name |hangul=햇...,{{nofootnote}} {{Infobox Korean name jeongch...


In [67]:
df_wiki_filtered = df_wiki_filtered.to_csv('20230914_filtered_wiki.csv', index = False, encoding = 'utf-8-sig')

In [71]:
#csv파일 합치기
df_1 = pd.read_csv('filtered_wiki_sum\\20230822_filtered_wikidata_sum.csv')
df_2 = pd.read_csv('20230914_filtered_wiki.csv')

df = pd.concat([df_1, df_2], ignore_index = True)
df

,title,date,text,cleansing_test
0,Konfusianisme,2023-07-17T03:00:00Z,{{tak netral}} {{rapikan}} {{Infobox Chinese |...,| suz Zyú-ka | j Jyu4-gaa1 | y Yùh-gāa |...
1,Asia,2023-07-19T12:11:07Z,{{kegunaanlain}} {{Infobox continent |title = ...,|population 4.641.054.775 (1) |density 87 ...
2,Kanji,2022-12-21T05:17:27Z,{{disambiginfo}} {{Infobox Writing system |nam...,"Zhuyin, Hanzi sederhana, Chu Nom, Aksara Khi..."
3,Aksara Han,2022-03-04T08:32:23Z,{{Infobox writing system |name=Aksara Han <br>...,"200px |piccap ""Aksara Han"" dalam bentuk tradi..."
4,Biksu,2023-05-29T06:39:19Z,{{more citations needed|date=Januari 2023}} [[...,Para bhikkhu sedang berdoa di Candi Borobudur...
...,...,...,...,...
9229,Temptation of a Wife,2023-02-11T22:21:49Z,{{penghubung}} * [[Temptation of Wife (serial ...,{{penghubung}} * [[Temptation of Wife of Wife'...
9230,Seven (penyanyi),2023-02-05T16:08:15Z,{{Infobox musical artist |Name = Se7en |Img = ...,= Seven sebagai model di sampul majalah KoreAm...
9231,Renault Samsung Motors,2022-12-29T14:34:54Z,{{Short description|Produsen mobil asal Korea ...,generasi ketiga ]] * [[Renault Samsung SM3 * [...
9232,Anapji,2022-01-10T12:24:04Z,{{Kotak info danau | lake_name = Anapji | imag...,</gallery> == Referensi == {{reflist}} == Pran...


In [72]:
df = df.to_csv('20230914_filtered_wikidata_sum.csv', index = False, encoding = 'utf-8-sig')
#--20230914--

In [26]:
import re
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import stanza

# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')

2023-10-12 15:13:54 INFO: Downloading default packages for language: id (Indonesian) ...
2023-10-12 15:13:55 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-10-12 15:13:57 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-10-12 15:13:57 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-12 15:13:57 WARNING: Language id package default expects mwt, which has been added
2023-10-12 15:13:57 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-10-12 15:13:57 INFO: Using device: cpu
2023-10-12 15:13:57 INFO: Loading: tokenize
2023-10-12 15:13:57 INFO: Loading: mwt
2023-10-12 15:13:57 INFO: Done loading processors!


In [27]:
df = pd.read_csv('C:\\Users\\USER\\Desktop\\20230927_wikidata_원천데이터_주한서_검수완료_2차검수완료.csv')
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29.0,upright1.15 Konfusianisme umumnya memang ti...,upright1.15 Konfusianisme umumnya memang tidak...,upright1.15 Konfusianisme umumnya memang tidak...,"['upright1.15', 'Konfusianisme', 'umumnya', 'm..."
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15.0,upright1.15 Konfusianisme umumnya memang ti...,Namun di Indonesia Konghucu diakui sebagai sal...,Namun di Indonesia Konghucu diakui sebagai sal...,"['Namun', 'di', 'Indonesia', 'Konghucu', 'diak..."
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13.0,upright1.15 Konfusianisme umumnya memang ti...,Konghucu sebagai agama digagas Kang Youwei men...,Konghucu sebagai agama digagas Kang Youwei men...,"['Konghucu', 'sebagai', 'agama', 'digagas', 'K..."
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35.0,upright1.15 Konfusianisme umumnya memang ti...,Namun gagasan Kang Youwei tampaknya tidak dite...,Namun gagasan Kang Youwei tampaknya tidak dite...,"['Namun', 'gagasan', 'Kang', 'Youwei', 'tampak..."
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37.0,upright1.15 Konfusianisme umumnya memang ti...,"Dalam bahasa Tionghoa, ajaran Konghucu dikenal...","Dalam bahasa Tionghoa , ajaran Konghucu dikena...","['Dalam', 'bahasa', 'Tionghoa', ',', 'ajaran',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66490,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,39.0,Sejong Center adalah sebuah kompleks seni ...,Balai Agung Sejong dirancang pada awalnya bag...,Balai Agung Sejong dirancang pada awalnya bagi...,"['Balai', 'Agung', 'Sejong', 'dirancang', 'pad..."
66491,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,26.0,Sejong Center adalah sebuah kompleks seni ...,"Sejak kehadirannya, Sejong Center telah member...","Sejak kehadirannya , Sejong Center telah membe...","['Sejak', 'kehadirannya', ',', 'Sejong', 'Cent..."
66492,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,28.0,Sejong Center adalah sebuah kompleks seni ...,Ukuran Balai Agung secara alami mengikuti jeni...,Ukuran Balai Agung secara alami mengikuti jeni...,"['Ukuran', 'Balai', 'Agung', 'secara', 'alami'..."
66493,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,39.0,Sejong Center adalah sebuah kompleks seni ...,Seoul Philharmonic Seoul National Philharmoni...,Seoul Philharmonic Seoul National Philharmonic...,"['Seoul', 'Philharmonic', 'Seoul', 'National',...

In [28]:
def process_test(text):

    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    #text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text) #wiki 점 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    #text = re.sub(r"\|.*? |\|\d+px", "", text) #remove | and image px
    text = re.sub(r'jmpl\|', '', text)
    text = re.sub(r'ka\|', '', text)
    text = re.sub(r'\d+px\|', '', text)
    text = re.sub(r'kiri\|', '', text)
    text = re.sub(r'thumb\|', '', text)
    text = re.sub(r'lurus\|', '', text)
    text = re.sub(r'left\|', '', text)
    text = re.sub(r'right\|', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"",text)
    text = re.sub(r"\|.*?   |\|\d+px","",text)
    text = re.sub(r"\|.*?  |\|\d+px","",text)
    text = re.sub(r"\|.*? |\|\d+px","",text)
    text = re.sub(r"\|.*?|\|\d+px","",text)
    text = re.sub(r"\d+px", "",text) #remove image px
    text = re.sub(r"[가-힣]+","",text) #remove 한글
    text = re.sub(r"[一-龥]+","",text) #remove 한자
    text = re.sub(r"\([^)]*\)","",text) #remove 소괄호, 소괄호 속 내용
    text = re.sub(r"http\S+", "", text) #remove http
    text = re.sub(r"\!","",text)
    text = re.sub(r'.*?\.jpg\s*', '', text) #jpg 앞쪽 문장 모두 제거
    text = re.sub(r'jpg', '', text)
    text = re.sub(r'Berkas:.*',"",text) #Berkas: 로 문장 시작 시 모두 제거
    text = re.sub(r'\s*section at the bottom of the .*$','',text)
    text = re.sub(r'text = re.sub(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF『』、。《》]", "", text)','',text) #remove 일본어
    text = re.sub(r'-->',"",text)
    text = re.sub(r'-',"",text)
    text = re.sub(r'::',"",text)
    text = re.sub(r'《',"",text)
    text = re.sub(r'》',"",text)
    text = re.sub(r'『',"",text)
    text = re.sub(r'』',"",text)
    text = re.sub(r'&x7C;',"",text)
    

    return text

Wiki 문장 분리 후 정제

In [29]:
# NaN 값을 처리합니다. 예를 들어, NaN 값을 빈 문자열로 대체합니다.
df['Sentence'] = df['Sentence'].fillna('')

# 모든 값을 문자열로 변환합니다.
df['Sentence'] = df['Sentence'].astype(str)


In [30]:
df['Applied sentence'] = df['Sentence'].apply(process_test)
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token,Applied sentence
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29.0,upright1.15 Konfusianisme umumnya memang ti...,upright1.15 Konfusianisme umumnya memang tidak...,upright1.15 Konfusianisme umumnya memang tidak...,"['upright1.15', 'Konfusianisme', 'umumnya', 'm...",upright1.15 Konfusianisme umumnya memang tidak...
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15.0,upright1.15 Konfusianisme umumnya memang ti...,Namun di Indonesia Konghucu diakui sebagai sal...,Namun di Indonesia Konghucu diakui sebagai sal...,"['Namun', 'di', 'Indonesia', 'Konghucu', 'diak...",Namun di Indonesia Konghucu diakui sebagai sal...
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13.0,upright1.15 Konfusianisme umumnya memang ti...,Konghucu sebagai agama digagas Kang Youwei men...,Konghucu sebagai agama digagas Kang Youwei men...,"['Konghucu', 'sebagai', 'agama', 'digagas', 'K...",Konghucu sebagai agama digagas Kang Youwei men...
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35.0,upright1.15 Konfusianisme umumnya memang ti...,Namun gagasan Kang Youwei tampaknya tidak dite...,Namun gagasan Kang Youwei tampaknya tidak dite...,"['Namun', 'gagasan', 'Kang', 'Youwei', 'tampak...",Namun gagasan Kang Youwei tampaknya tidak dite...
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37.0,upright1.15 Konfusianisme umumnya memang ti...,"Dalam bahasa Tionghoa, ajaran Konghucu dikenal...","Dalam bahasa Tionghoa , ajaran Konghucu dikena...","['Dalam', 'bahasa', 'Tionghoa', ',', 'ajaran',...","Dalam bahasa Tionghoa, ajaran Konghucu dikenal..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66490,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,39.0,Sejong Center adalah sebuah kompleks seni ...,Balai Agung Sejong dirancang pada awalnya bag...,Balai Agung Sejong dirancang pada awalnya bagi...,"['Balai', 'Agung', 'Sejong', 'dirancang', 'pad...",Balai Agung Sejong dirancang pada awalnya bag...
66491,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,26.0,Sejong Center adalah sebuah kompleks seni ...,"Sejak kehadirannya, Sejong Center telah member...","Sejak kehadirannya , Sejong Center telah membe...","['Sejak', 'kehadirannya', ',', 'Sejong', 'Cent...","Sejak kehadirannya, Sejong Center telah member..."
66492,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,28.0,Sejong Center adalah sebuah kompleks seni ...,Ukuran Balai Agung secara alami mengikuti jeni...,Ukuran Balai Agung secara alami mengikuti jeni...,"['Ukuran', 'Balai', 'Agung', 'secara', 'alami'...",Ukuran Balai Agung secara alami mengikuti jeni...
66493,20230816_wikidata_

In [31]:
#'Sentence' column에 'Applied_Sentence' column을 적용
df['Sentence'] =df['Applied sentence']

In [32]:
#drop 'Applied_Sentence'
df = df.drop(columns=['Applied sentence'])
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29.0,upright1.15 Konfusianisme umumnya memang ti...,upright1.15 Konfusianisme umumnya memang tidak...,upright1.15 Konfusianisme umumnya memang tidak...,"['upright1.15', 'Konfusianisme', 'umumnya', 'm..."
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15.0,upright1.15 Konfusianisme umumnya memang ti...,Namun di Indonesia Konghucu diakui sebagai sal...,Namun di Indonesia Konghucu diakui sebagai sal...,"['Namun', 'di', 'Indonesia', 'Konghucu', 'diak..."
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13.0,upright1.15 Konfusianisme umumnya memang ti...,Konghucu sebagai agama digagas Kang Youwei men...,Konghucu sebagai agama digagas Kang Youwei men...,"['Konghucu', 'sebagai', 'agama', 'digagas', 'K..."
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35.0,upright1.15 Konfusianisme umumnya memang ti...,Namun gagasan Kang Youwei tampaknya tidak dite...,Namun gagasan Kang Youwei tampaknya tidak dite...,"['Namun', 'gagasan', 'Kang', 'Youwei', 'tampak..."
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37.0,upright1.15 Konfusianisme umumnya memang ti...,"Dalam bahasa Tionghoa, ajaran Konghucu dikenal...","Dalam bahasa Tionghoa , ajaran Konghucu dikena...","['Dalam', 'bahasa', 'Tionghoa', ',', 'ajaran',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66490,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,39.0,Sejong Center adalah sebuah kompleks seni ...,Balai Agung Sejong dirancang pada awalnya bag...,Balai Agung Sejong dirancang pada awalnya bagi...,"['Balai', 'Agung', 'Sejong', 'dirancang', 'pad..."
66491,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,26.0,Sejong Center adalah sebuah kompleks seni ...,"Sejak kehadirannya, Sejong Center telah member...","Sejak kehadirannya , Sejong Center telah membe...","['Sejak', 'kehadirannya', ',', 'Sejong', 'Cent..."
66492,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,28.0,Sejong Center adalah sebuah kompleks seni ...,Ukuran Balai Agung secara alami mengikuti jeni...,Ukuran Balai Agung secara alami mengikuti jeni...,"['Ukuran', 'Balai', 'Agung', 'secara', 'alami'..."
66493,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,39.0,Sejong Center adalah sebuah kompleks seni ...,Seoul Philharmonic Seoul National Philharmoni...,Seoul Philharmonic Seoul National Philharmonic...,"['Seoul', 'Philharmonic', 'Seoul', 'National',...

In [33]:
#정규표현식을 적용한 'Sentence' column에 있는 텍스트를 토크나이징
def tokenizing_sent(df):
    new_rows = []
    max_length = 5
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        article_sent = row['Sentence']
        stanza_doc = nlp(article_sent)
        tokens = [token.text for sent in stanza_doc.sentences for token in sent.tokens]
        new_row = row.copy()
        new_row['Token']= tokens
        new_row['Word_Count'] = len(tokens)
        new_rows.append(new_row)
    
    new_df = pd.DataFrame(new_rows)
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)

    new_df.drop_duplicates(subset='Sentence', inplace=True)
    
    new_df.reset_index(drop=True, inplace=True)
    new_df.rename(columns={'Source_File' : 'Filename', 'Headline':'Title', 'Keyword':'Pub_Subj', 'Date': 'Pub_Date'}, inplace=True)
    
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df

In [34]:
#'Sentence' column의 null값이 아닌 것들만 추출
df =df[~df['Sentence'].isnull()]
df.reset_index(drop=True, inplace=True)

In [35]:
df = tokenizing_sent(df)
df

Processing rows: 100%|██████████| 66495/66495 [12:44<00:00, 86.99it/s] 


,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,Sentence,Tokenized_Sentence,Token
0,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000001,29,upright1.15 Konfusianisme umumnya memang ti...,upright1.15 Konfusianisme umumnya memang tidak...,upright1.15 Konfusianisme umumnya memang tidak...,"[upright1.15, Konfusianisme, umumnya, memang, ..."
1,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000002,15,upright1.15 Konfusianisme umumnya memang ti...,Namun di Indonesia Konghucu diakui sebagai sal...,Namun di Indonesia Konghucu diakui sebagai sal...,"[Namun, di, Indonesia, Konghucu, diakui, sebag..."
2,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000003,13,upright1.15 Konfusianisme umumnya memang ti...,Konghucu sebagai agama digagas Kang Youwei men...,Konghucu sebagai agama digagas Kang Youwei men...,"[Konghucu, sebagai, agama, digagas, Kang, Youw..."
3,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000004,35,upright1.15 Konfusianisme umumnya memang ti...,Namun gagasan Kang Youwei tampaknya tidak dite...,Namun gagasan Kang Youwei tampaknya tidak dite...,"[Namun, gagasan, Kang, Youwei, tampaknya, tida..."
4,20230816_wikidata_Konfusianisme_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Konfusianisme,Wikipedia,Konfusianisme,2023-07-17,2023-08-10,20230816_wikidata_Konfusianisme_000001_sen000005,37,upright1.15 Konfusianisme umumnya memang ti...,"Dalam bahasa Tionghoa, ajaran Konghucu dikenal...","Dalam bahasa Tionghoa , ajaran Konghucu dikena...","[Dalam, bahasa, Tionghoa, ,, ajaran, Konghucu,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66488,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,64,Sejong Center adalah sebuah kompleks seni ...,Dibangun di lokasi yang sebelumnya merupakan S...,Dibangun di lokasi yang sebelumnya merupakan S...,"[Dibangun, di, lokasi, yang, sebelumnya, merup..."
66489,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,39,Sejong Center adalah sebuah kompleks seni ...,Balai Agung Sejong dirancang pada awalnya bag...,Balai Agung Sejong dirancang pada awalnya bagi...,"[Balai, Agung, Sejong, dirancang, pada, awalny..."
66490,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,26,Sejong Center adalah sebuah kompleks seni ...,"Sejak kehadirannya, Sejong Center telah member...","Sejak kehadirannya , Sejong Center telah membe...","[Sejak, kehadirannya, ,, Sejong, Center, telah..."
66491,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Pusat Seni Pertunjukan Sejong,Wikipedia,Pusat Seni Pertunjukan Sejong,2022-11-23,2023-08-10,20230816_wikidata_Pusat Seni Pertunjukan Sejon...,28,Sejong Center adalah sebuah kompleks seni ...,Ukuran Balai Agung secara alami mengikuti jeni...,Ukuran Balai Agung secara alami mengikuti jeni...,"[Ukuran, Balai, Agung, secara, alami, mengikut..."


In [36]:
df['Word_Count'].sum()

1655492

In [37]:
# 토큰별 작업용 컬럼 생성
for i in range(df['Word_Count'].max()):
    new_column_name = f'Column_{i}'  # 새 열 이름 생성
    df[new_column_name]=np.nan

for idx in tqdm(df.index):
    for i, token in enumerate(df['Token'][idx]):
        new_column_name = f'Column_{i}'  # 새 열 이름 생성
        df[new_column_name][idx]=token

C:\Users\USER\AppData\Local\Temp\ipykernel_11508\2584547734.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name]=np.nan
C:\Users\USER\AppData\Local\Temp\ipykernel_11508\2584547734.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_column_name]=np.nan
C:\Users\USER\AppData\Local\Temp\ipykernel_11508\2584547734.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [38]:
# 'Sen_ID'기준으로 정렬

df.sort_values('Sen_ID', inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Doc_ID,Filename,Title,Pub_Type,Pub_Subj,Pub_Date,Col_Date,Sen_ID,Word_Count,Text,...,Column_2744,Column_2745,Column_2746,Column_2747,Column_2748,Column_2749,Column_2750,Column_2751,Column_2752,Column_2753
0,"20230816_wikidata_""O""-Jung.Ban.Hap._000001",C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,"""O""-Jung.Ban.Hap.",Wikipedia,"""O""-Jung.Ban.Hap.",2022-10-28,2023-08-10,"20230816_wikidata_""O""-Jung.Ban.Hap._000001_sen...",25,"""O""-... adalah album studio ketiga oleh boyb...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"20230816_wikidata_""O""-Jung.Ban.Hap._000001",C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,"""O""-Jung.Ban.Hap.",Wikipedia,"""O""-Jung.Ban.Hap.",2022-10-28,2023-08-10,"20230816_wikidata_""O""-Jung.Ban.Hap._000001_sen...",29,"""O""-... adalah album studio ketiga oleh boyb...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"20230816_wikidata_""O""-Jung.Ban.Hap._000001",C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,"""O""-Jung.Ban.Hap.",Wikipedia,"""O""-Jung.Ban.Hap.",2022-10-28,2023-08-10,"20230816_wikidata_""O""-Jung.Ban.Hap._000001_sen...",13,"""O""-... adalah album studio ketiga oleh boyb...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"20230816_wikidata_""O""-Jung.Ban.Hap._000001",C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,"""O""-Jung.Ban.Hap.",Wikipedia,"""O""-Jung.Ban.Hap.",2022-10-28,2023-08-10,"20230816_wikidata_""O""-Jung.Ban.Hap._000001_sen...",23,"""O""-... adalah album studio ketiga oleh boyb...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"20230816_wikidata_""O""-Jung.Ban.Hap._000001",C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,"""O""-Jung.Ban.Hap.",Wikipedia,"""O""-Jung.Ban.Hap.",2022-10-28,2023-08-10,"20230816_wikidata_""O""-Jung.Ban.Hap._000001_sen...",25,"""O""-... adalah album studio ketiga oleh boyb...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66478,20230816_wikidata_Zico (penyanyi rap)_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Zico (penyanyi rap),Wikipedia,Zico (penyanyi rap),2023-06-13,2023-08-10,20230816_wikidata_Zico (penyanyi rap)_000001_s...,32,"Zico lahir di Mapo, Seoul pada 14 September...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66479,20230816_wikidata_Zico (penyanyi rap)_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Zico (penyanyi rap),Wikipedia,Zico (penyanyi rap),2023-06-13,2023-08-10,20230816_wikidata_Zico (penyanyi rap)_000001_s...,34,"Zico lahir di Mapo, Seoul pada 14 September...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66480,20230816_wikidata_Zico (penyanyi rap)_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Zico (penyanyi rap),Wikipedia,Zico (penyanyi rap),2023-06-13,2023-08-10,20230816_wikidata_Zico (penyanyi rap)_000001_s...,27,"Zico lahir di Mapo, Seoul pada 14 September...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66481,20230816_wikidata_Zico (penyanyi rap)_000001,C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NI...,Zico (penyanyi rap),Wikipedia,Zico (penyanyi rap),2023-06-13,2023-08-10,20230816_wikidata_Zico (penyanyi rap)_000001_s...,13,"Zico lahir di Mapo, Seoul pada 14 September...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터'

In [42]:
df.to_csv('20231012_원천데이터(위키)_정제완료.csv', index=False, encoding='utf-8-sig')
#end of tokenizing

Wiki 문장 분리 전 정제

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('C:\\Users\\USER\\Desktop\\wiki 한국-인도네시아 관련 표제어231024.csv')
df

,title,date,text,cleansing_test,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 177,Unnamed: 178,Unnamed: 179,Unnamed: 180,Unnamed: 181,Unnamed: 182,Unnamed: 183,Unnamed: 184,Unnamed: 185,Unnamed: 186
0,My Lovely Liar,2023-07-18T07:39:07Z,{{Infobox television | image = My Lovely Liar....,| genre | creator | developer Studio Drag...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dreamers (lagu Jungkook),2023-07-20T09:46:24Z,{{Infobox song | name = Dreamers | image = Dre...,| released | studio Katara Studios (Doha) ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Woo Jin-young,2023-02-04T18:44:15Z,{{Orphan|date=Februari 2023}} {{Family name ha...,| birth_place | death_date | death_place ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Heo Sung-hyun,2023-01-28T03:03:48Z,{{Infobox musical artist | name = Heo Sung-hyu...,| birth_name Heo Sung-hyun | birth_date | ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Louie (penyanyi rap),2023-06-20T16:03:57Z,{{family name hatnote|Hwang||lang=Korean}} {{I...,2021|access-date=August 8,2021|language=ko}}</ref> ==Referensi== {{refl...,"| birth_place Seoul, Korea Selatan | spouse...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Kwon Hyun-bin,2022-12-01T07:42:23Z,{{Korean name|Kwon}} {{Infobox person | name =...,2017|work=Osen|language=ko}}</ref> == Filmogr...,珥ъ쁺 ?꾩옣 鍮꾪븯?몃뱶 鸚㏐났媛?date=December 18,2017|work=Ten Asia|language=ko|access-date=20...,4??tvN D ?밸뱶?쇰쭏 濡좎묶??븘由?沅뚰쁽鍮?異쒖뿰[怨듭떇?낆옣]|date...,2020|work=SPOTV News|language=ko}}</ref> |- |...,2020|work=entertain.v.daum.net|language=ko}}<...,"2020|work=MK|language=ko}}</ref> |- ! scope=""...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,Melting Me Softly,2023-06-21T21:26:45Z,{{Infobox television | show_name = Melting Me ...,| genre FantasiKomedi romantis | creator St...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,Crash Landing on You,2023-06-09T09:34:23Z,{{Infobox television | show_name = Crash Landi...,| native_name | genre Komedi romantis | cr...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,AfreecaTV,2022-07-20T14:13:40Z,{{lowercase title}} {{infobox dot-com company ...,"| num_employees | parent AfreecaTV Co., Lt...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def process_test(text):

    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    #text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text) #wiki 점 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    #text = re.sub(r"\|.*? |\|\d+px", "", text) #remove | and image px
    text = re.sub(r'jmpl\|', '', text)
    text = re.sub(r'ka\|', '', text)
    text = re.sub(r'\d+px\|', '', text)
    text = re.sub(r'kiri\|', '', text)
    text = re.sub(r'thumb\|', '', text)
    text = re.sub(r'lurus\|', '', text)
    text = re.sub(r'left\|', '', text)
    text = re.sub(r'right\|', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"",text)
    text = re.sub(r"\|.*?   |\|\d+px","",text)
    text = re.sub(r"\|.*?  |\|\d+px","",text)
    text = re.sub(r"\|.*? |\|\d+px","",text)
    text = re.sub(r"\|.*?|\|\d+px","",text)
    text = re.sub(r"\d+px", "",text) #remove image px
    text = re.sub(r"[가-힣]+","",text) #remove 한글
    text = re.sub(r"[一-龥]+","",text) #remove 한자
    text = re.sub(r"\([^)]*\)","",text) #remove 소괄호, 소괄호 속 내용
    text = re.sub(r"http\S+", "", text) #remove http
    text = re.sub(r"\!","",text)
    text = re.sub(r'.*?\.jpg\s*', '', text) #jpg 앞쪽 문장 모두 제거
    text = re.sub(r'jpg', '', text)
    text = re.sub(r'Berkas:.*',"",text) #Berkas: 로 문장 시작 시 모두 제거
    text = re.sub(r'\s*section at the bottom of the .*$','',text)
    text = re.sub(r'text = re.sub(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF『』、。《》]", "", text)','',text) #remove 일본어
    text = re.sub(r'-->',"",text)
    text = re.sub(r'-',"",text)
    text = re.sub(r'::',"",text)
    text = re.sub(r'《',"",text)
    text = re.sub(r'》',"",text)
    text = re.sub(r'『',"",text)
    text = re.sub(r'』',"",text)
    text = re.sub(r'&x7C;',"",text)
    

    return text

In [4]:
df = df.loc[:,:'text']
df

,title,date,text
0,My Lovely Liar,2023-07-18T07:39:07Z,{{Infobox television | image = My Lovely Liar....
1,Dreamers (lagu Jungkook),2023-07-20T09:46:24Z,{{Infobox song | name = Dreamers | image = Dre...
2,Woo Jin-young,2023-02-04T18:44:15Z,{{Orphan|date=Februari 2023}} {{Family name ha...
3,Heo Sung-hyun,2023-01-28T03:03:48Z,{{Infobox musical artist | name = Heo Sung-hyu...
4,Louie (penyanyi rap),2023-06-20T16:03:57Z,{{family name hatnote|Hwang||lang=Korean}} {{I...
...,...,...,...
310,Kwon Hyun-bin,2022-12-01T07:42:23Z,{{Korean name|Kwon}} {{Infobox person | name =...
311,Melting Me Softly,2023-06-21T21:26:45Z,{{Infobox television | show_name = Melting Me ...
312,Crash Landing on You,2023-06-09T09:34:23Z,{{Infobox television | show_name = Crash Landi...
313,AfreecaTV,2022-07-20T14:13:40Z,{{lowercase title}} {{infobox dot-com company ...


In [5]:
df['cleansing_text'] = df['text'].apply(process_test)
df

C:\Users\USER\AppData\Local\Temp\ipykernel_12500\1503964798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleansing_text'] = df['text'].apply(process_test)


,title,date,text,cleansing_text
0,My Lovely Liar,2023-07-18T07:39:07Z,{{Infobox television | image = My Lovely Liar....,My Lovely Liar adalah seri televisi Korea S...
1,Dreamers (lagu Jungkook),2023-07-20T09:46:24Z,{{Infobox song | name = Dreamers | image = Dre...,3:21 3:59 Jungkook Mustapha El Ouardi Pat...
2,Woo Jin-young,2023-02-04T18:44:15Z,{{Orphan|date=Februari 2023}} {{Family name ha...,Woo Jinyoung adalah penyanyi dan penyanyi...
3,Heo Sung-hyun,2023-01-28T03:03:48Z,{{Infobox musical artist | name = Heo Sung-hyu...,Heo Sunghyun juga dikenal dengan Huh atau ...
4,Louie (penyanyi rap),2023-06-20T16:03:57Z,{{family name hatnote|Hwang||lang=Korean}} {{I...,"Hwang Munseob , better known by his stage n..."
...,...,...,...,...
310,Kwon Hyun-bin,2022-12-01T07:42:23Z,{{Korean name|Kwon}} {{Infobox person | name =...,"Kwon Hyunbin adalah seorang model, rapper d..."
311,Melting Me Softly,2023-06-21T21:26:45Z,{{Infobox television | show_name = Melting Me ...,Melting Me Softly adalah seri televisi Kore...
312,Crash Landing on You,2023-06-09T09:34:23Z,{{Infobox television | show_name = Crash Landi...,Crash Landing on You adalah seri televisi K...
313,AfreecaTV,2022-07-20T14:13:40Z,{{lowercase title}} {{infobox dot-com company ...,860 AfreecaTV adalah sebuah layanan penyi...


In [7]:
df = df.to_csv('filterwiki_문장분리전_20231024(효진님).csv', index = False, encoding = 'utf-8-sig')

In [1]:
import os
os.getcwd()

'c:\\Users\\USER\\Ars Praxia\\[T] 23-05-001 NIA AI 데이터셋 구축 - General\\06-수집데이터\\위키 데이터'

## wikidata 문장분리 및 정제 20231030

In [6]:
#wikidata assemble 20231030
import pandas as pd
import numpy as np
import os

In [14]:
path = 'C:\\Users\\USER\\Desktop\\wiki_koreadata_all\\wiki_data_all_korea\\wiki_final_원천데이터_all'
os.chdir(path)

In [11]:
os.getcwd()

'C:\\Users\\USER\\Desktop\\wiki_koreadata_all\\wiki_data_all_korea'

In [15]:
file_lists = os.listdir()
data_frame = []

for file_list in file_lists:
    data_frame.append(pd.read_csv(os.path.join(path, file_list)))

df = pd.concat(data_frame)
df

C:\Users\USER\AppData\Local\Temp\ipykernel_19856\1652965620.py:5: DtypeWarning: Columns (161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388

,title,date,text,cleansing_text,cleansing_test,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 1290,Unnamed: 1291,Unnamed: 1292,Unnamed: 1293,Unnamed: 1294,Unnamed: 1295,Unnamed: 1296,Unnamed: 1297,Unnamed: 1298,Unnamed: 1299
0,Suneung,2023-06-22T18:10:28Z,{{Infobox Korean name| hangul=??숈닔?숇뒫?μ떆?? ha...,Ujian Kemampuan Skolastik ke Perguruan Tingg...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lee Dong-won,2023-03-03T14:46:52Z,{{Infobox football biography |name = Lee Dong-...,Lee DongWon adalah pemain sepak bola Korea S...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jun Ji-hyun,2022-11-30T20:16:25Z,{{Korean name|Wang|Jun}} {{Infobox person | na...,Seoul at Daum,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hyuna,2022-11-29T09:26:39Z,{{Korean name|Kim}} {{Infobox person | name = ...,Kim Hyunah atau lebih dikenal dengan monon...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pyongyang (jaringan restoran),2022-12-11T10:24:43Z,{{otheruses|Pyongyang}} [[Berkas:Pyongyang Res...,Restoran Pyongyang di Phnom Penh Pyongyang ad...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2113,Oasis (seri televisi Korea Selatan),2023-07-09 12:29,{{Infobox television | image = Oasis (South Ko...,NaN,| genre | writer Jung Hyung-soo | director...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2114,Templat:Infobox Korean,2023-07-11 14:50,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,NaN,| | yes | | | | | yes | | |headercolor...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2115,Daftar katedral di Korea Utara,2023-07-18 20:17,[[Berkas:Jangchung Catholic Church (1552450067...,NaN,"Katedral Changchung di Pyongyang, Korea Utara....",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2116,Banjir Korea Selatan 2023,2023-07-18 3:10,{{current disaster|date=Juli 2023}} {{Infobox ...,NaN,) | damages | missing 9 | fatalities 41 | ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df = df.drop_duplicates(subset = 'title')

In [19]:
df.reset_index(drop=True, inplace=True)
df.dropna()
df

,title,date,text,cleansing_text,cleansing_test,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 1290,Unnamed: 1291,Unnamed: 1292,Unnamed: 1293,Unnamed: 1294,Unnamed: 1295,Unnamed: 1296,Unnamed: 1297,Unnamed: 1298,Unnamed: 1299
0,Suneung,2023-06-22T18:10:28Z,{{Infobox Korean name| hangul=??숈닔?숇뒫?μ떆?? ha...,Ujian Kemampuan Skolastik ke Perguruan Tingg...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lee Dong-won,2023-03-03T14:46:52Z,{{Infobox football biography |name = Lee Dong-...,Lee DongWon adalah pemain sepak bola Korea S...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jun Ji-hyun,2022-11-30T20:16:25Z,{{Korean name|Wang|Jun}} {{Infobox person | na...,Seoul at Daum,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hyuna,2022-11-29T09:26:39Z,{{Korean name|Kim}} {{Infobox person | name = ...,Kim Hyunah atau lebih dikenal dengan monon...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pyongyang (jaringan restoran),2022-12-11T10:24:43Z,{{otheruses|Pyongyang}} [[Berkas:Pyongyang Res...,Restoran Pyongyang di Phnom Penh Pyongyang ad...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12108,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,NaN,| genre | writer Jung Hyung-soo | director...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12109,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,NaN,| | yes | | | | | yes | | |headercolor...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12110,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...,NaN,"Katedral Changchung di Pyongyang, Korea Utara....",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12111,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...,NaN,) | damages | missing 9 | fatalities 41 | ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df = df.to_csv('wikiall원천.csv', index = False, encoding = 'utf-8-sig')

In [21]:
df = pd.read_csv('wikiall원천.csv')
df

C:\Users\USER\AppData\Local\Temp\ipykernel_19856\2231681735.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256

,title,date,text,cleansing_text,cleansing_test,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,...,Unnamed: 1290,Unnamed: 1291,Unnamed: 1292,Unnamed: 1293,Unnamed: 1294,Unnamed: 1295,Unnamed: 1296,Unnamed: 1297,Unnamed: 1298,Unnamed: 1299
0,Suneung,2023-06-22T18:10:28Z,{{Infobox Korean name| hangul=??숈닔?숇뒫?μ떆?? ha...,Ujian Kemampuan Skolastik ke Perguruan Tingg...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Lee Dong-won,2023-03-03T14:46:52Z,{{Infobox football biography |name = Lee Dong-...,Lee DongWon adalah pemain sepak bola Korea S...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jun Ji-hyun,2022-11-30T20:16:25Z,{{Korean name|Wang|Jun}} {{Infobox person | na...,Seoul at Daum,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hyuna,2022-11-29T09:26:39Z,{{Korean name|Kim}} {{Infobox person | name = ...,Kim Hyunah atau lebih dikenal dengan monon...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Pyongyang (jaringan restoran),2022-12-11T10:24:43Z,{{otheruses|Pyongyang}} [[Berkas:Pyongyang Res...,Restoran Pyongyang di Phnom Penh Pyongyang ad...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12108,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,NaN,| genre | writer Jung Hyung-soo | director...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12109,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,NaN,| | yes | | | | | yes | | |headercolor...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12110,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...,NaN,"Katedral Changchung di Pyongyang, Korea Utara....",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12111,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...,NaN,) | damages | missing 9 | fatalities 41 | ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def process_test(text):

    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"&lt;", "<", text)
    text = re.sub(r"&gt;", ">", text)
    text = re.sub(r"<ref[^<]*</ref>", "", text)
    text = re.sub(r"\[\[[^\|\]]*\|", "[[", text)
    text = re.sub(r"{{[^}]*}}", "", text)
    text = re.sub(r"{[^}]*}", "", text)
    text = re.sub(r"<[^>]*>", "", text)
    #text = re.sub(r"\|\d+px", "", text) #remove image px
    text = re.sub(r"==\s*([^=]+)\s*=+", "", text) #소제목 제거
    text = re.sub(r"\=", "", text)
    text = re.sub(r"\[http[^\]]+\]", "", text) #http 제거
    text = re.sub(r"\[\[Kategori:[^\]]+\]\]","", text) #카테고리 제거
    text = re.sub(r"\{", "", text)
    text = re.sub(r"\}", "", text)
    text = re.sub(r"\\'","", text) #wiki에서 글씨체 효과 제거
    text = re.sub(r"\'''", "", text) #wiki에서 bold처리 표현식 제거
    text = re.sub(r"\''", "", text) #wiki에서 이탤릭체 표현식 제거
    text = re.sub(r"\#", "", text) #wiki 숫자 표시 제거
    text = re.sub(r"\*", "", text) #wiki 점 제거
    text = re.sub(r"\[", "", text)
    text = re.sub(r"\]", "", text)
    #text = re.sub(r"\|[^|]+\|", "|", text)
    #text = re.sub(r"\|.*? |\|\d+px", "", text) #remove | and image px
    text = re.sub(r'jmpl\|', '', text)
    text = re.sub(r'ka\|', '', text)
    text = re.sub(r'\d+px\|', '', text)
    text = re.sub(r'kiri\|', '', text)
    text = re.sub(r'thumb\|', '', text)
    text = re.sub(r'lurus\|', '', text)
    text = re.sub(r'left\|', '', text)
    text = re.sub(r'right\|', '', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',"",text)
    text = re.sub(r"\|.*?   |\|\d+px","",text)
    text = re.sub(r"\|.*?  |\|\d+px","",text)
    text = re.sub(r"\|.*? |\|\d+px","",text)
    text = re.sub(r"\|.*?|\|\d+px","",text)
    text = re.sub(r"\d+px", "",text) #remove image px
    text = re.sub(r"[가-힣]+","",text) #remove 한글
    text = re.sub(r"[一-龥]+","",text) #remove 한자
    text = re.sub(r"\([^)]*\)","",text) #remove 소괄호, 소괄호 속 내용
    text = re.sub(r"http\S+", "", text) #remove http
    text = re.sub(r"\!","",text)
    text = re.sub(r'.*?\.jpg\s*', '', text) #jpg 앞쪽 문장 모두 제거
    text = re.sub(r'jpg', '', text)
    text = re.sub(r'Berkas:.*',"",text) #Berkas: 로 문장 시작 시 모두 제거
    text = re.sub(r'\s*section at the bottom of the .*$','',text)
    text = re.sub(r'text = re.sub(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF『』、。《》]", "", text)','',text) #remove 일본어
    text = re.sub(r'-->',"",text)
    text = re.sub(r'-',"",text)
    text = re.sub(r'::',"",text)
    text = re.sub(r'《',"",text)
    text = re.sub(r'》',"",text)
    text = re.sub(r'『',"",text)
    text = re.sub(r'』',"",text)
    text = re.sub(r'&x7C;',"",text)
    

    return text

In [23]:
df = df.loc[:,:'text']
df

,title,date,text
0,Suneung,2023-06-22T18:10:28Z,{{Infobox Korean name| hangul=??숈닔?숇뒫?μ떆?? ha...
1,Lee Dong-won,2023-03-03T14:46:52Z,{{Infobox football biography |name = Lee Dong-...
2,Jun Ji-hyun,2022-11-30T20:16:25Z,{{Korean name|Wang|Jun}} {{Infobox person | na...
3,Hyuna,2022-11-29T09:26:39Z,{{Korean name|Kim}} {{Infobox person | name = ...
4,Pyongyang (jaringan restoran),2022-12-11T10:24:43Z,{{otheruses|Pyongyang}} [[Berkas:Pyongyang Res...
...,...,...,...
12108,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...
12109,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...
12110,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...
12111,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...


In [25]:
# NaN 값을 처리합니다. 예를 들어, NaN 값을 빈 문자열로 대체합니다.
df['text'] = df['text'].fillna('')

# 모든 값을 문자열로 변환합니다.
df['text'] = df['text'].astype(str)


In [26]:
df['cleansing_text'] = df['text'].apply(process_test)
df

,title,date,text,cleansing_text
0,Suneung,2023-06-22T18:10:28Z,{{Infobox Korean name| hangul=??숈닔?숇뒫?μ떆?? ha...,Ujian Kemampuan Skolastik ke Perguruan Tingg...
1,Lee Dong-won,2023-03-03T14:46:52Z,{{Infobox football biography |name = Lee Dong-...,Lee DongWon adalah pemain sepak bola Korea S...
2,Jun Ji-hyun,2022-11-30T20:16:25Z,{{Korean name|Wang|Jun}} {{Infobox person | na...,Seoul at Daum
3,Hyuna,2022-11-29T09:26:39Z,{{Korean name|Kim}} {{Infobox person | name = ...,Kim Hyunah atau lebih dikenal dengan monon...
4,Pyongyang (jaringan restoran),2022-12-11T10:24:43Z,{{otheruses|Pyongyang}} [[Berkas:Pyongyang Res...,Restoran Pyongyang di Phnom Penh Pyongyang ad...
...,...,...,...,...
12108,Oasis (seri televisi Korea Selatan),2023-07-09T03:29:01Z,{{Infobox television | image = Oasis (South Ko...,Oasis adalah seri televisi Korea Selatan ta...
12109,Templat:Infobox Korean,2023-07-11T05:50:00Z,{{#if:{{{hangul|}}}{{{hanja|}}}{{{rr|}}}{{{mr|...,
12110,Daftar katedral di Korea Utara,2023-07-18T11:17:41Z,[[Berkas:Jangchung Catholic Church (1552450067...,"Katedral Changchung di Pyongyang, Korea Utara...."
12111,Banjir Korea Selatan 2023,2023-07-17T18:10:10Z,{{current disaster|date=Juli 2023}} {{Infobox ...,) Curah hujan yang tinggi selama musim hujan ...


In [29]:
import re
import pandas as pd
import os
import stanza
# Stanza 초기화
stanza.download('id')  # 영어 모델 다운로드
nlp = stanza.Pipeline('id', processors='tokenize')
from tqdm import tqdm

2023-10-30 21:00:38 INFO: Downloading default packages for language: id (Indonesian) ...
2023-10-30 21:00:39 INFO: File exists: C:\Users\USER\stanza_resources\id\default.zip
2023-10-30 21:00:42 INFO: Finished downloading models and saved to C:\Users\USER\stanza_resources.
2023-10-30 21:00:42 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-30 21:00:43 WARNING: Language id package default expects mwt, which has been added
2023-10-30 21:00:43 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |

2023-10-30 21:00:43 INFO: Using device: cpu
2023-10-30 21:00:43 INFO: Loading: tokenize
2023-10-30 21:00:43 INFO: Loading: mwt
2023-10-30 21:00:43 INFO: Done loading processors!


In [30]:
def tokenizing(df):
    max_length=5
    new_rows = []
    index=0
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
        index+=1
        article_text = row['cleansing_text']
        doc = nlp(article_text)
        sen_id = 1  # 문장 ID를 1로 초기화
        for sentence in doc.sentences:
            new_row = row.copy()
            new_row['Sentence'] = sentence.text
            new_row['Token']= [token.text for token in sentence.tokens]
            new_row['Word_Count'] = len(new_row['Token'])
            new_row['Doc_ID']='20230816_wikidata_'+new_row['title']+'_000001'
            new_row['Sen_ID'] = new_row['Doc_ID']+'_sen'+str('{:06d}'.format(sen_id))
            new_row['Pub_Date']=new_row['date'][:10]
            new_row['Title']=new_row['title']
            new_row['Pub_Subj']=new_row['title']
            new_rows.append(new_row)
            sen_id += 1  # 문장 ID 증가

    new_df = pd.DataFrame(new_rows)
    new_df['Tokenized_Sentence']= new_df['Token'].apply(' '.join)
    new_df['Col_Date']='2023-08-10'
    new_df['Pub_Type']='Wikipedia'
    new_df['Text']=new_df['cleansing_text']
    new_df['Filename']='C:/Users/Juhee Kim/Ars Praxia/[T] 23-05-001 NIA AI 데이터셋 구축 - General/06-수집데이터/위키 데이터/rawdata/idwiki-20230720-pages-articles-multistream.xml'

    new_df.drop_duplicates(subset='Sentence', inplace=True)
    new_df.reset_index(drop=True, inplace=True)
    new_df = new_df[['Doc_ID', 'Filename', 'Title', 'Pub_Type', 'Pub_Subj', 'Pub_Date', 'Col_Date', 'Sen_ID', 'Word_Count', 'Text', 'Sentence','Tokenized_Sentence' , 'Token']]
    new_df = new_df[new_df['Token'].apply(lambda x: len(x) > max_length)]
    return new_df

In [31]:
df_test= tokenizing(df)
df_test

Processing rows:   1%|          | 81/12113 [00:06<15:21, 13.05it/s] 


KeyboardInterrupt: 

### 테스트웍스 전달 시 작업용 column 사용

In [ ]:
# 토큰별 작업용 컬럼 생성
for i in range(df['Word_Count'].max()):
    new_column_name = f'Column_{i}'  # 새 열 이름 생성
    df[new_column_name]=np.nan

for idx in tqdm(df.index):
    for i, token in enumerate(df['Token'][idx]):
        new_column_name = f'Column_{i}'  # 새 열 이름 생성
        df[new_column_name][idx]=token

In [ ]:
# 'Sen_ID'기준으로 정렬

df.sort_values('Sen_ID', inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
new_df = df_test.to_csv('wikiall원천20231030.csv', index=False, encoding = 'utf-8-sig')